In [59]:
#install.packages("processR")
#install.packages("lavaan") 

import os
import numpy as np
import pandas as pd
import statsmodels.api as sm




## Donations

In [60]:
#paths to exeriments

INPUT_PATH = "../../../Merging/Results/master_with_dv_means_recoded.csv"

import os

def make_path(exp_code: str, filename: str = "merged.csv") -> str:
    """
    Build the path directly from the experiment code.
    Folders match the code exactly.
    
    """
    base_dir = "../../../Analysis/Results/Per_Experiment_Recoded"
    return os.path.join(base_dir, exp_code, filename)




## Donation_Money

In [61]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm


# ============================================================
# 1. Setup and load data
# ============================================================

INPUT_PATH = "../../../Merging/Results/master_with_dv_means_recoded.csv"
OUTPUT_DIR_MALL = "Tables/Mall/Spendentafel/Donation_Money/Model4"
OUTPUT_DIR_BEACHBAR = "Tables/BeachBar/Spendentafel/Donation_Money/Model4"

os.makedirs(OUTPUT_DIR_MALL, exist_ok=True)
os.makedirs(OUTPUT_DIR_BEACHBAR, exist_ok=True)


df = pd.read_csv(INPUT_PATH)

# Filter: Mall donation money experiment only (M_D_M)
mall_dm = df[df["Experiment_Type"] == "M_D_M"].copy()

beachBar_dm = df[df["Experiment_Type"] == "BB_T_D_M"].copy()

print(f"Sample size (Mall Donation Money, M_D_M): {len(mall_dm)}")


# ============================================================
# 2. Helper functions
# ============================================================

def sig_label(pval: float) -> str:
    if pval < 0.05:
        return "Significant"
    elif pval < 0.10:
        return "Marginally significant"
    else:
        return "Not significant"


def format_p(p: float) -> str:
    """APA-like p-value formatting."""
    if p < 0.001:
        return "< .001"
    return f"= {p:.4f}".replace("= 0.", "= .")


# ============================================================
# 3. Simple mediation (PROCESS Model 4) with tables + console output
# ============================================================

def run_mediation_and_export(
    data: pd.DataFrame,
    x_col: str,
    m_col: str,
    y_col: str,
    outfile_prefix: str,
    output_dir: str,
    n_boot: int = 5000,
    seed: int = 42,
):
    rng = np.random.default_rng(seed)
    sub = data[[x_col, m_col, y_col]].dropna().copy()
    n = len(sub)

    print(f"\n▶ Running mediation: Mediator = {m_col}, n = {n}")

    # -------------------------
    # Path a (M ~ X)
    # -------------------------
    X_a = sm.add_constant(sub[x_col])

    model_a = sm.OLS(sub[m_col], X_a).fit()

    a_coef = round(model_a.params[x_col], 4)
    a_se = round(model_a.bse[x_col], 4)
    a_t = round(model_a.tvalues[x_col], 4)
    a_p = round(model_a.pvalues[x_col], 4)
    a_ll, a_ul = model_a.conf_int().loc[x_col].round(4)

    # -------------------------
    # Paths b and c' (Y ~ X + M)
    # -------------------------
    X_b = sm.add_constant(sub[[x_col, m_col]])

    # path c and c'
    model_b = sm.OLS(sub[y_col], X_b).fit()

    b_coef = round(model_b.params[m_col], 4)
    b_se = round(model_b.bse[m_col], 4)
    b_t = round(model_b.tvalues[m_col], 4)
    b_p = round(model_b.pvalues[m_col], 4)
    b_ll, b_ul = model_b.conf_int().loc[m_col].round(4)

    cprime_coef = round(model_b.params[x_col], 4)
    cprime_se = round(model_b.bse[x_col], 4)
    cprime_t = round(model_b.tvalues[x_col], 4)
    cprime_p = round(model_b.pvalues[x_col], 4)
    cprime_ll, cprime_ul = model_b.conf_int().loc[x_col].round(4)

    # -------------------------
    # Total effect c (Y ~ X)
    # -------------------------
    X_c = sm.add_constant(sub[x_col])
    model_c = sm.OLS(sub[y_col], X_c).fit()

    c_coef = round(model_c.params[x_col], 4)
    c_se = round(model_c.bse[x_col], 4)
    c_t = round(model_c.tvalues[x_col], 4)
    c_p = round(model_c.pvalues[x_col], 4)
    c_ll, c_ul = model_c.conf_int().loc[x_col].round(4)

    # -------------------------
    # Indirect effect
    # -------------------------
    indirect = round(a_coef * b_coef, 4)

    boot_vals = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        boot = sub.iloc[idx]

        X_a_b = sm.add_constant(boot[x_col])
        a_b = sm.OLS(boot[m_col], X_a_b).fit().params[x_col]

        X_b_b = sm.add_constant(boot[[x_col, m_col]])
        b_b = sm.OLS(boot[y_col], X_b_b).fit().params[m_col]

        boot_vals.append(a_b * b_b)

    boot_vals = np.array(boot_vals)
    ci_low, ci_high = np.percentile(boot_vals, [2.5, 97.5]).round(4)
    boot_se = round(boot_vals.std(), 4)
    indirect_sig = "Significant" if (ci_low > 0 or ci_high < 0) else "Not significant"

    # ============================================================
    # Save tables with 4 decimals
    # ============================================================

    results_paths = pd.DataFrame({
        "Path": ["Total Effect (c)", "Direct Effect (c')", "Path a (X → M)", "Path b (M → Y | X)"],
        "Coefficient (B)": [c_coef, cprime_coef, a_coef, b_coef],
        "SE": [c_se, cprime_se, a_se, b_se],
        "t-value": [c_t, cprime_t, a_t, b_t],
        "p-value": [c_p, cprime_p, a_p, b_p],
        "95% CI (LL)": [c_ll, cprime_ll, a_ll, b_ll],
        "95% CI (UL)": [c_ul, cprime_ul, a_ul, b_ul],
        "Significance": [
            sig_label(c_p), sig_label(cprime_p),
            sig_label(a_p), sig_label(b_p)
        ]
    })

    results_indirect = pd.DataFrame({
        "Effect (a × b)": [indirect],
        "Boot SE": [boot_se],
        "BootLLCI": [ci_low],
        "BootULCI": [ci_high],
        "Significance": [indirect_sig],
    })

    path_outfile = os.path.join(output_dir, f"{outfile_prefix}_paths.csv")
    indirect_outfile = os.path.join(output_dir, f"{outfile_prefix}_indirect.csv")

    results_paths.to_csv(path_outfile, float_format="%.4f", index=False)
    results_indirect.to_csv(indirect_outfile, float_format="%.4f", index=False)

    print(f"Saved path table     → {path_outfile}")
    print(f"Saved indirect table → {indirect_outfile}")

    # ============================================================
    # Console output (4 decimals)
    # ============================================================

    print("\nAPA-style summary")
    print("-----------------")
    print(f"Mediator: {m_col}")
    print(f"n = {n}")

    print(
        f"Total effect (c): b = {c_coef:.4f}, SE = {c_se:.4f}, "
        f"t({int(model_c.df_resid)}) = {c_t:.4f}, p {format_p(c_p)}, "
        f"95% CI [{c_ll:.4f}, {c_ul:.4f}]"
    )

    print(
        f"Direct effect (c'): b = {cprime_coef:.4f}, SE = {cprime_se:.4f}, "
        f"t({int(model_b.df_resid)}) = {cprime_t:.4f}, p {format_p(cprime_p)}, "
        f"95% CI [{cprime_ll:.4f}, {cprime_ul:.4f}]"
    )

    print(
        f"Path a (X → M): b = {a_coef:.4f}, SE = {a_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a_t:.4f}, p {format_p(a_p)}, "
        f"95% CI [{a_ll:.4f}, {a_ul:.4f}]"
    )

    print(
        f"Path b (M → Y | X): b = {b_coef:.4f}, SE = {b_se:.4f}, "
        f"t({int(model_b.df_resid)}) = {b_t:.4f}, p {format_p(b_p)}, "
        f"95% CI [{b_ll:.4f}, {b_ul:.4f}]"
    )

    print(
        f"Indirect effect (a × b): {indirect:.4f}, Boot SE = {boot_se:.4f}, "
        f"95% Boot CI [{ci_low:.4f}, {ci_high:.4f}] → {indirect_sig}"
    )

    return results_paths, results_indirect


# ============================================================
# 4. Run mediation for OLD and NEW status scales
# ============================================================

X = "Style_Glam0_Hippie1"
Y = "DonationMoney"


run_mediation_and_export(
    data=mall_dm,
    x_col=X,
    m_col="Social_Status_Old_1_4",
    y_col=Y,
    outfile_prefix="Donation_Money_Status_Old",
    output_dir = OUTPUT_DIR_MALL
)


run_mediation_and_export(
    data=mall_dm,
    x_col=X,
    m_col="Social_Status_New_1_7",
    y_col=Y,
    outfile_prefix="M_Donation_Money_Status_New",
    output_dir = OUTPUT_DIR_MALL
)

##for beach beach trash donation money 


print("BeachBar Info")

run_mediation_and_export(
    data=beachBar_dm,
    x_col=X,
    m_col="Social_Status_Old_1_4",
    y_col=Y,
    outfile_prefix="M_Donation_Money_Status_Old",
    output_dir = OUTPUT_DIR_BEACHBAR
)

run_mediation_and_export(
    data=beachBar_dm,
    x_col=X,
    m_col="Social_Status_New_1_7",
    y_col=Y,
    outfile_prefix="BB_Donation_Money_Status_New",
    output_dir = OUTPUT_DIR_BEACHBAR
)
""" 
##beach bar donation money
Y1 = "trashCount"


run_mediation_and_export(
    data=beachBar_dm,
    x_col=X,
    m_col="Social_Status_Old_1_4",
    y_col=Y1,
    outfile_prefix="mediation_task1_status_new",
    output_dir = OUTPUT_DIR_BEACHBAR
)

run_mediation_and_export(
    data=beachBar_dm,
    x_col=X,
    m_col="Social_Status_New_1_7",
    y_col=Y1,
    outfile_prefix="mediation_task1_status_new",
    output_dir = OUTPUT_DIR_BEACHBAR
) """


print("\n✅ Mediation Task 1 completed and tables exported.")


Sample size (Mall Donation Money, M_D_M): 119

▶ Running mediation: Mediator = Social_Status_Old_1_4, n = 119
Saved path table     → Tables/Mall/Spendentafel/Donation_Money/Model4/Donation_Money_Status_Old_paths.csv
Saved indirect table → Tables/Mall/Spendentafel/Donation_Money/Model4/Donation_Money_Status_Old_indirect.csv

APA-style summary
-----------------
Mediator: Social_Status_Old_1_4
n = 119
Total effect (c): b = -15.6790, SE = 7.5501, t(117) = -2.0767, p = .0400, 95% CI [-30.6315, -0.7265]
Direct effect (c'): b = -13.8320, SE = 7.6580, t(116) = -1.8062, p = .0735, 95% CI [-28.9996, 1.3357]
Path a (X → M): b = -0.3562, SE = 0.1758, t(117) = -2.0262, p = .0450, 95% CI [-0.7045, -0.0080]
Path b (M → Y | X): b = 5.1846, SE = 3.9578, t(116) = 1.3100, p = .1928, 95% CI [-2.6544, 13.0236]
Indirect effect (a × b): -1.8468, Boot SE = 2.0064, 95% Boot CI [-6.6771, 1.2669] → Not significant

▶ Running mediation: Mediator = Social_Status_New_1_7, n = 119
Saved path table     → Tables/Mall/

## Process Model 7 - Donation Money with Moderator Spendentafel on A Path

In [62]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ============================================================
# 0. Settings
# ============================================================

OUTPUT_DIR_M  = "Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model7"
OUTPUT_DIR_BB = "Tables/BeachBar/Spendentafel/Donation_Money_Spendentafel/Model7"

X_COL = "Style_Glam0_Hippie1"            # Glam = 0, Hippie = 1
Y_COL = "DonationMoney"
W_COL = "Visibility_0_Private_1_Public"  # 0 = Private, 1 = Public


# ============================================================
# 1. Load and merge Mall + BeachBar data
# ============================================================
# NOTE: assumes `make_path(exp_code, filename="merged.csv")` exists.

# --- Mall: Donation Money with / without Spendentafel ---
path_plain_M = make_path("M_D_M")
path_spend_M = make_path("M_D_S_M")

df_plain_M = pd.read_csv(path_plain_M)
df_spend_M = pd.read_csv(path_spend_M)

mall_dm = pd.concat([df_plain_M, df_spend_M], ignore_index=True)

# --- BeachBar: Donation Money with / without Spendentafel ---
path_plain_BB = make_path("BB_T_D_M")
path_spend_BB = make_path("BB_T_D_SM")

df_plain_BB = pd.read_csv(path_plain_BB)
df_spend_BB = pd.read_csv(path_spend_BB)

beachbar_dm = pd.concat([df_plain_BB, df_spend_BB], ignore_index=True)

# ============================================================
# 2. Ensure visibility dummy + keep only required columns
# ============================================================

def ensure_visibility_dummy(df):
    if W_COL in df.columns:
        return df
    if "Visibility" in df.columns:
        df[W_COL] = df["Visibility"].map({"Private": 0, "Public": 1})
        return df
    raise ValueError(
        f"Neither {W_COL} nor 'Visibility' column found in data."
    )

mall_dm = ensure_visibility_dummy(mall_dm)
beachbar_dm = ensure_visibility_dummy(beachbar_dm)

needed_cols = [
    X_COL,
    Y_COL,
    W_COL,
    "Social_Status_Old_1_4",
    "Social_Status_New_1_7",
]

mall_dm = mall_dm[needed_cols].dropna().copy()
beachbar_dm = beachbar_dm[needed_cols].dropna().copy()

print(
    f"Sample size (Mall Donation Money with/without Spendentafel): "
    f"{len(mall_dm)}"
)
print(
    f"Sample size (BeachBar Donation Money with/without Spendentafel): "
    f"{len(beachbar_dm)}"
)

# ============================================================
# 3. PROCESS Model 7 (a-path moderated) + export
# ============================================================

def run_process_model7_and_export(
    data: pd.DataFrame,
    x_col: str,
    m_col: str,
    y_col: str,
    w_col: str,
    dir: str,
    outfile_prefix: str,
    n_boot: int = 5000,
    seed: int = 42,
):

    os.makedirs(dir, exist_ok=True)

    rng = np.random.default_rng(seed)

    sub = data[[x_col, m_col, y_col, w_col]].dropna().copy()
    n = len(sub)

    # Interaction term X*W
    sub["XW"] = sub[x_col] * sub[w_col]

    print(f"\n▶ Running PROCESS Model 7: Mediator = {m_col}, n = {n}")

    # -------------------------
    # a-path (moderated): M ~ X + W + XW
    # -------------------------
    Xa = sm.add_constant(sub[[x_col, w_col, "XW"]])
    model_a = sm.OLS(sub[m_col], Xa).fit()

    a1 = round(model_a.params[x_col], 4)      # X
    a2 = round(model_a.params[w_col], 4)      # W
    a3 = round(model_a.params["XW"], 4)       # interaction

    a1_se = round(model_a.bse[x_col], 4)
    a2_se = round(model_a.bse[w_col], 4)
    a3_se = round(model_a.bse["XW"], 4)

    a1_t = round(model_a.tvalues[x_col], 4)
    a2_t = round(model_a.tvalues[w_col], 4)
    a3_t = round(model_a.tvalues["XW"], 4)

    a1_p = round(model_a.pvalues[x_col], 4)
    a2_p = round(model_a.pvalues[w_col], 4)
    a3_p = round(model_a.pvalues["XW"], 4)

    a1_ll, a1_ul = model_a.conf_int().loc[x_col].round(4)
    a2_ll, a2_ul = model_a.conf_int().loc[w_col].round(4)
    a3_ll, a3_ul = model_a.conf_int().loc["XW"].round(4)

    # -------------------------
    # Y model (b and c'): Y ~ X + M + W
    # -------------------------
    Xy = sm.add_constant(sub[[x_col, m_col, w_col]])
    model_y = sm.OLS(sub[y_col], Xy).fit()

    b = round(model_y.params[m_col], 4)       # M → Y
    d = round(model_y.params[w_col], 4)       # W → Y
    cprime = round(model_y.params[x_col], 4)  # X → Y (direct)

    b_se = round(model_y.bse[m_col], 4)
    d_se = round(model_y.bse[w_col], 4)
    cprime_se = round(model_y.bse[x_col], 4)

    b_t = round(model_y.tvalues[m_col], 4)
    d_t = round(model_y.tvalues[w_col], 4)
    cprime_t = round(model_y.tvalues[x_col], 4)

    b_p = round(model_y.pvalues[m_col], 4)
    d_p = round(model_y.pvalues[w_col], 4)
    cprime_p = round(model_y.pvalues[x_col], 4)

    b_ll, b_ul = model_y.conf_int().loc[m_col].round(4)
    d_ll, d_ul = model_y.conf_int().loc[w_col].round(4)
    cprime_ll, cprime_ul = model_y.conf_int().loc[x_col].round(4)

    # -------------------------
    # Total effect c: Y ~ X + W
    # -------------------------
    Xt = sm.add_constant(sub[[x_col, w_col]])
    model_c = sm.OLS(sub[y_col], Xt).fit()

    c = round(model_c.params[x_col], 4)
    c_se = round(model_c.bse[x_col], 4)
    c_t = round(model_c.tvalues[x_col], 4)
    c_p = round(model_c.pvalues[x_col], 4)
    c_ll, c_ul = model_c.conf_int().loc[x_col].round(4)

    # -------------------------
    # Conditional indirect effects
    # -------------------------
    # W = 0 (Private): a1 * b
    indirect_0 = round(a1 * b, 4)
    # W = 1 (Public): (a1 + a3) * b
    indirect_1 = round((a1 + a3) * b, 4)

    # Bootstrap CI for both conditional indirects
    boot_0 = []
    boot_1 = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        boot = sub.iloc[idx].copy()
        boot["XW"] = boot[x_col] * boot[w_col]

        Xa_b = sm.add_constant(boot[[x_col, w_col, "XW"]])
        a_b = sm.OLS(boot[m_col], Xa_b).fit()

        Xy_b = sm.add_constant(boot[[x_col, m_col, w_col]])
        y_b = sm.OLS(boot[y_col], Xy_b).fit()

        a1_b = a_b.params[x_col]
        a3_b = a_b.params["XW"]
        b_b = y_b.params[m_col]

        boot_0.append(a1_b * b_b)
        boot_1.append((a1_b + a3_b) * b_b)

    boot_0 = np.array(boot_0)
    boot_1 = np.array(boot_1)

    ci0_low, ci0_high = np.percentile(boot_0, [2.5, 97.5]).round(4)
    ci1_low, ci1_high = np.percentile(boot_1, [2.5, 97.5]).round(4)

    se0 = round(boot_0.std(), 4)
    se1 = round(boot_1.std(), 4)

    sig0 = "Significant" if (ci0_low > 0 or ci0_high < 0) else "Not significant"
    sig1 = "Significant" if (ci1_low > 0 or ci1_high < 0) else "Not significant"

    # ========================================================
    # Build tables
    # ========================================================

    # Path coefficients
    results_paths = pd.DataFrame({
        "Path": [
            "Total Effect (c)",
            "Direct Effect (c')",
            "Path a1 (X → M)",
            "Path a2 (W → M)",
            "Path a3 (X×W → M)",
            "Path b (M → Y | X,W)",
            "Path d (W → Y | X,M)",
        ],
        "Coefficient (B)": [c, cprime, a1, a2, a3, b, d],
        "SE": [c_se, cprime_se, a1_se, a2_se, a3_se, b_se, d_se],
        "t-value": [c_t, cprime_t, a1_t, a2_t, a3_t, b_t, d_t],
        "p-value": [c_p, cprime_p, a1_p, a2_p, a3_p, b_p, d_p],
        "95% CI (LL)": [c_ll, cprime_ll, a1_ll, a2_ll, a3_ll, b_ll, d_ll],
        "95% CI (UL)": [c_ul, cprime_ul, a1_ul, a2_ul, a3_ul, b_ul, d_ul],
    })

    results_paths["Significance"] = results_paths["p-value"].apply(sig_label)

    # Conditional indirect effects
    results_indirect = pd.DataFrame({
        "Moderator_Value": [0, 1],
        "Moderator_Label": ["Private", "Public"],
        "Effect (a-path moderated)": [indirect_0, indirect_1],
        "Boot SE": [se0, se1],
        "BootLLCI": [ci0_low, ci1_low],
        "BootULCI": [ci0_high, ci1_high],
        "Significance": [sig0, sig1],
    })

    path_outfile = os.path.join(dir, f"{outfile_prefix}_paths.csv")
    indirect_outfile = os.path.join(dir, f"{outfile_prefix}_indirect.csv")

    results_paths.to_csv(path_outfile, float_format="%.4f", index=False)
    results_indirect.to_csv(indirect_outfile, float_format="%.4f", index=False)

    print(f"Saved path table     → {path_outfile}")
    print(f"Saved indirect table → {indirect_outfile}")

    # ========================================================
    # Console output (APA-style)
    # ========================================================

    print("\nAPA-style summary")
    print("-----------------")
    print(f"Mediator: {m_col}")
    print(f"n = {n}")

    print(
        f"Total effect (c): b = {c:.4f}, SE = {c_se:.4f}, "
        f"t({int(model_c.df_resid)}) = {c_t:.4f}, p {format_p(c_p)}, "
        f"95% CI [{c_ll:.4f}, {c_ul:.4f}]"
    )

    print(
        f"Direct effect (c'): b = {cprime:.4f}, SE = {cprime_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {cprime_t:.4f}, p {format_p(cprime_p)}, "
        f"95% CI [{cprime_ll:.4f}, {cprime_ul:.4f}]"
    )

    print(
        f"Path a1 (X → M): b = {a1:.4f}, SE = {a1_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a1_t:.4f}, p {format_p(a1_p)}, "
        f"95% CI [{a1_ll:.4f}, {a1_ul:.4f}]"
    )

    print(
        f"Path a2 (W → M): b = {a2:.4f}, SE = {a2_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a2_t:.4f}, p {format_p(a2_p)}, "
        f"95% CI [{a2_ll:.4f}, {a2_ul:.4f}]"
    )

    print(
        f"Path a3 (X×W → M): b = {a3:.4f}, SE = {a3_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a3_t:.4f}, p {format_p(a3_p)}, "
        f"95% CI [{a3_ll:.4f}, {a3_ul:.4f}]"
    )

    print(
        f"Path b (M → Y | X,W): b = {b:.4f}, SE = {b_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b_t:.4f}, p {format_p(b_p)}, "
        f"95% CI [{b_ll:.4f}, {b_ul:.4f}]"
    )

    print(
        f"Path d (W → Y | X,M): b = {d:.4f}, SE = {d_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {d_t:.4f}, p {format_p(d_p)}, "
        f"95% CI [{d_ll:.4f}, {d_ul:.4f}]"
    )

    print("\nConditional indirect effects (moderated mediation via a-path):")
    print(
        f"- Visibility = 0 (Private): "
        f"Effect = {indirect_0:.4f}, Boot SE = {se0:.4f}, "
        f"95% CI [{ci0_low:.4f}, {ci0_high:.4f}] → {sig0}"
    )
    print(
        f"- Visibility = 1 (Public):  "
        f"Effect = {indirect_1:.4f}, Boot SE = {se1:.4f}, "
        f"95% CI [{ci1_low:.4f}, {ci1_high:.4f}] → {sig1}"
    )

    return results_paths, results_indirect

# ============================================================
# 4. Run for OLD and NEW status scales (Mall + BeachBar)
# ============================================================

# Mall
run_process_model7_and_export(
    data=mall_dm,
    x_col=X_COL,
    m_col="Social_Status_Old_1_4",
    y_col=Y_COL,
    w_col=W_COL,
    dir=OUTPUT_DIR_M,
    outfile_prefix="M_Donation_Spendentafel_Status_Old",
)

run_process_model7_and_export(
    data=mall_dm,
    x_col=X_COL,
    m_col="Social_Status_New_1_7",
    y_col=Y_COL,
    w_col=W_COL,
    dir=OUTPUT_DIR_M,
    outfile_prefix="M_Donation_Spendentafel_Status_New",
)

# BeachBar
run_process_model7_and_export(
    data=beachbar_dm,
    x_col=X_COL,
    m_col="Social_Status_Old_1_4",
    y_col=Y_COL,
    w_col=W_COL,
    dir=OUTPUT_DIR_BB,
    outfile_prefix="BB_Donation_Spendentafel_Status_Old",
)

run_process_model7_and_export(
    data=beachbar_dm,
    x_col=X_COL,
    m_col="Social_Status_New_1_7",
    y_col=Y_COL,
    w_col=W_COL,
    dir=OUTPUT_DIR_BB,
    outfile_prefix="BB_Donation_Spendentafel_Status_New",
)

print("\n✅ PROCESS Model 7 (Spendentafel as a-path moderator) completed for Mall and BeachBar and tables exported.")


Sample size (Mall Donation Money with/without Spendentafel): 119
Sample size (BeachBar Donation Money with/without Spendentafel): 158

▶ Running PROCESS Model 7: Mediator = Social_Status_Old_1_4, n = 119
Saved path table     → Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model7/M_Donation_Spendentafel_Status_Old_paths.csv
Saved indirect table → Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model7/M_Donation_Spendentafel_Status_Old_indirect.csv

APA-style summary
-----------------
Mediator: Social_Status_Old_1_4
n = 119
Total effect (c): b = -15.6790, SE = 7.5501, t(117) = -2.0767, p = .0400, 95% CI [-30.6315, -0.7265]
Direct effect (c'): b = -13.8320, SE = 7.6580, t(116) = -1.8062, p = .0735, 95% CI [-28.9996, 1.3357]
Path a1 (X → M): b = -0.3562, SE = 0.1758, t(117) = -2.0262, p = .0450, 95% CI [-0.7045, -0.0080]
Path a2 (W → M): b = 0.0000, SE = 0.0000, t(117) = nan, p = nan, 95% CI [0.0000, 0.0000]
Path a3 (X×W → M): b = 0.0000, SE = 0.0000, t(117) = nan, p = nan, 95%

### On C Path Moderation Process Model 5

In [63]:

# ============================================================
# 0. Settings / Pfade
# ============================================================

BASE_DIR = "../../../Analysis/Results/Per_Experiment_Recoded"

def make_path(exp_code: str, filename: str = "merged.csv") -> str:
    return os.path.join(BASE_DIR, exp_code, filename)

OUTPUT_DIR_MALL  = "Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model5"
OUTPUT_DIR_BB    = "Tables/BeachBar/Spendentafel/Donation_Money_Spendentafel/Model5"

os.makedirs(OUTPUT_DIR_MALL, exist_ok=True)
os.makedirs(OUTPUT_DIR_BB, exist_ok=True)

X_COL = "Style_Glam0_Hippie1"           # Glam = 0, Hippie = 1
Y_COL = "DonationMoney"
W_COL = "Visibility_0_Private_1_Public" # 0 = Private, 1 = Public

N_BOOT = 5000
SEED   = 42

rng = np.random.default_rng(SEED)

# ============================================================
# 1. Helper functions
# ============================================================

def sig_label(p: float) -> str:
    if pd.isna(p):
        return "Not estimable"
    if p < 0.05:
        return "Significant"
    elif p < 0.10:
        return "Marginally significant"
    else:
        return "Not significant"

def format_p(p: float) -> str:
    if pd.isna(p):
        return "= n/a"
    if p < 0.001:
        return "< .001"
    s = f"= {p:.4f}"
    # APA: = .0123 statt = 0.0123
    return s.replace("= 0.", "= .")

# ============================================================
# 2. Kernfunktion: c-Pfad-Moderation (PROCESS Model 5-like)
#    a:  M ~ X
#    c' (moderiert): Y ~ X + M + W + X*W
#    c  (moderiert): Y ~ X + W + X*W
#    Indirekter Effekt: a * b, mit Bootstrap-CI
# ============================================================

def run_model5_moderation_and_export(
    data: pd.DataFrame,
    x_col: str,
    m_col: str,
    y_col: str,
    w_col: str,
    out_dir: str,
    outfile_prefix: str,
    n_boot: int = 5000,
    rng: np.random.Generator = rng,
):
    # nur benötigte Spalten, listwise deletion
    sub = data[[x_col, m_col, y_col, w_col]].dropna().copy()
    n = len(sub)

    # Interaktion X*W
    sub["XW"] = sub[x_col] * sub[w_col]

    print(f"\n▶ Running c-path moderation (PROCESS-style Model 5): Mediator = {m_col}, n = {n}")

    # -------------------------
    # a-Pfad: M ~ X  (unmoderiert)
    # -------------------------
    X_a = sm.add_constant(sub[[x_col]])
    model_a = sm.OLS(sub[m_col], X_a).fit()

    a = model_a.params[x_col]
    a_se = model_a.bse[x_col]
    a_t = model_a.tvalues[x_col]
    a_p = model_a.pvalues[x_col]
    a_ll, a_ul = model_a.conf_int().loc[x_col]
    df_a = int(model_a.df_resid)

    # -------------------------
    # Y-Modell (c' moderiert): Y ~ X + M + W + X*W
    # -------------------------
    X_y = sm.add_constant(sub[[x_col, m_col, w_col, "XW"]])
    model_y = sm.OLS(sub[y_col], X_y).fit()

    # b-Pfad (M -> Y)
    b = model_y.params[m_col]
    b_se = model_y.bse[m_col]
    b_t = model_y.tvalues[m_col]
    b_p = model_y.pvalues[m_col]
    b_ll, b_ul = model_y.conf_int().loc[m_col]

    # d-Pfad (W -> Y | X,M,XW)
    d = model_y.params[w_col]
    d_se = model_y.bse[w_col]
    d_t = model_y.tvalues[w_col]
    d_p = model_y.pvalues[w_col]
    d_ll, d_ul = model_y.conf_int().loc[w_col]

    # c'-Haupteffekt X
    c_main = model_y.params[x_col]
    c_main_se = model_y.bse[x_col]
    c_main_t = model_y.tvalues[x_col]
    c_main_p = model_y.pvalues[x_col]
    c_main_ll, c_main_ul = model_y.conf_int().loc[x_col]

    # c'-Interaktion X:W (Name ist "XW")
    c_int = model_y.params["XW"]
    c_int_se = model_y.bse["XW"]
    c_int_t = model_y.tvalues["XW"]
    c_int_p = model_y.pvalues["XW"]
    c_int_ll, c_int_ul = model_y.conf_int().loc["XW"]

    df_y = int(model_y.df_resid)

    # bedingte direkte Effekte c'
    cprime_private = c_main              # W = 0
    cprime_public  = c_main + c_int      # W = 1

    # -------------------------
    # Total effect (c moderiert): Y ~ X + W + X*W
    # -------------------------
    X_c = sm.add_constant(sub[[x_col, w_col, "XW"]])
    model_c = sm.OLS(sub[y_col], X_c).fit()

    c_tot_main = model_c.params[x_col]
    c_tot_se_main = model_c.bse[x_col]
    c_tot_t_main = model_c.tvalues[x_col]
    c_tot_p_main = model_c.pvalues[x_col]
    c_tot_ll_main, c_tot_ul_main = model_c.conf_int().loc[x_col]

    c_tot_int = model_c.params["XW"]
    c_tot_se_int = model_c.bse["XW"]
    c_tot_t_int = model_c.tvalues["XW"]
    c_tot_p_int = model_c.pvalues["XW"]
    c_tot_ll_int, c_tot_ul_int = model_c.conf_int().loc["XW"]

    df_c = int(model_c.df_resid)

    # bedingte Total-Effekte
    c_tot_private = c_tot_main                 # W = 0
    c_tot_public  = c_tot_main + c_tot_int     # W = 1

    # -------------------------
    # Indirekter Effekt a*b (unmoderiert)
    # -------------------------
    indirect = a * b

    # Bootstrap für indirekten Effekt
    boot_vals = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        boot = sub.iloc[idx].copy()

        # a-Pfad im Bootstrap
        X_a_b = sm.add_constant(boot[[x_col]])
        a_b = sm.OLS(boot[m_col], X_a_b).fit().params[x_col]

        # Y-Modell im Bootstrap
        X_y_b = sm.add_constant(boot[[x_col, m_col, w_col, "XW"]])
        y_b = sm.OLS(boot[y_col], X_y_b).fit()
        b_b = y_b.params[m_col]

        boot_vals.append(a_b * b_b)

    boot_vals = np.array(boot_vals)
    boot_se = float(boot_vals.std(ddof=1))
    ci_low, ci_high = np.percentile(boot_vals, [2.5, 97.5])
    indirect_sig = "Significant" if (ci_low > 0 or ci_high < 0) else "Not significant"

    # ========================================================
    # 3. Tabellen bauen & speichern
    # ========================================================
    results_paths = pd.DataFrame({
        "Path": [
            "Total Effect main (c_X)",
            "Total Effect interaction (c_XW)",
            "Direct Effect main (c'_X)",
            "Direct Effect interaction (c'_XW)",
            "Path a (X → M)",
            "Path b (M → Y | X,W,XW)",
            "Path d (W → Y | X,M,XW)",
        ],
        "Coefficient (B)": [
            c_tot_main, c_tot_int,
            c_main, c_int,
            a, b, d
        ],
        "SE": [
            c_tot_se_main, c_tot_se_int,
            c_main_se, c_int_se,
            a_se, b_se, d_se
        ],
        "t-value": [
            c_tot_t_main, c_tot_t_int,
            c_main_t, c_int_t,
            a_t, b_t, d_t
        ],
        "p-value": [
            c_tot_p_main, c_tot_p_int,
            c_main_p, c_int_p,
            a_p, b_p, d_p
        ],
        "95% CI (LL)": [
            c_tot_ll_main, c_tot_ll_int,
            c_main_ll, c_int_ll,
            a_ll, b_ll, d_ll
        ],
        "95% CI (UL)": [
            c_tot_ul_main, c_tot_ul_int,
            c_main_ul, c_int_ul,
            a_ul, b_ul, d_ul
        ],
    })

    results_paths["Significance"] = results_paths["p-value"].apply(sig_label)

    results_indirect = pd.DataFrame({
        "Effect (a x b)": [indirect],
        "Boot SE": [boot_se],
        "BootLLCI": [ci_low],
        "BootULCI": [ci_high],
        "Significance": [indirect_sig],
    })

    os.makedirs(out_dir, exist_ok=True)
    path_outfile = os.path.join(out_dir, f"{outfile_prefix}_paths.csv")
    indirect_outfile = os.path.join(out_dir, f"{outfile_prefix}_indirect.csv")

    results_paths.to_csv(path_outfile, float_format="%.4f", index=False)
    results_indirect.to_csv(indirect_outfile, float_format="%.4f", index=False)

    print(f"Saved path table     → {path_outfile}")
    print(f"Saved indirect table → {indirect_outfile}")

    # ========================================================
    # 4. APA-Style Konsole
    # ========================================================

    print("\nAPA-style summary")
    print("-----------------")
    print(f"Mediator: {m_col}")
    print(f"n = {n}\n")

    # Total effect
    print(
        f"Total effect main (c_X): b = {c_tot_main:.4f}, SE = {c_tot_se_main:.4f}, "
        f"t({df_c}) = {c_tot_t_main:.4f}, p {format_p(c_tot_p_main)}, "
        f"95% CI [{c_tot_ll_main:.4f}, {c_tot_ul_main:.4f}]"
    )
    print(
        f"Total effect interaction (c_XW): b = {c_tot_int:.4f}, SE = {c_tot_se_int:.4f}, "
        f"t({df_c}) = {c_tot_t_int:.4f}, p {format_p(c_tot_p_int)}, "
        f"95% CI [{c_tot_ll_int:.4f}, {c_tot_ul_int:.4f}]"
    )
    print(
        f"Conditional total effects: W=0 (Private): c = {c_tot_private:.4f}, "
        f"W=1 (Public): c = {c_tot_public:.4f}"
    )
    print()

    # Direct effect
    print(
        f"Direct effect main (c'_X): b = {c_main:.4f}, SE = {c_main_se:.4f}, "
        f"t({df_y}) = {c_main_t:.4f}, p {format_p(c_main_p)}, "
        f"95% CI [{c_main_ll:.4f}, {c_main_ul:.4f}]"
    )
    print(
        f"Direct effect interaction (c'_XW): b = {c_int:.4f}, SE = {c_int_se:.4f}, "
        f"t({df_y}) = {c_int_t:.4f}, p {format_p(c_int_p)}, "
        f"95% CI [{c_int_ll:.4f}, {c_int_ul:.4f}]"
    )
    print(
        f"Conditional direct effects (c'): W=0 (Private): c' = {cprime_private:.4f}, "
        f"W=1 (Public): c' = {cprime_public:.4f}"
    )
    print()

    # a-, b-, d-Pfade
    print(
        f"Path a (X → M): b = {a:.4f}, SE = {a_se:.4f}, "
        f"t({df_a}) = {a_t:.4f}, p {format_p(a_p)}, "
        f"95% CI [{a_ll:.4f}, {a_ul:.4f}]"
    )
    print(
        f"Path b (M → Y | X,W,XW): b = {b:.4f}, SE = {b_se:.4f}, "
        f"t({df_y}) = {b_t:.4f}, p {format_p(b_p)}, "
        f"95% CI [{b_ll:.4f}, {b_ul:.4f}]"
    )
    print(
        f"Path d (W → Y | X,M,XW): b = {d:.4f}, SE = {d_se:.4f}, "
        f"t({df_y}) = {d_t:.4f}, p {format_p(d_p)}, "
        f"95% CI [{d_ll:.4f}, {d_ul:.4f}]"
    )
    print()

    # Indirekter Effekt
    print(
        f"Indirect effect (a x b): {indirect:.4f}, Boot SE = {boot_se:.4f}, "
        f"95% Boot CI [{ci_low:.4f}, {ci_high:.4f}] → {indirect_sig}"
    )

    return results_paths, results_indirect

# ============================================================
# 3. Daten laden: Mall + BeachBar
# ============================================================

# Mall: M_D_M vs M_D_S_M
path_mall_plain = make_path("M_D_M")
path_mall_spend = make_path("M_D_S_M")

df_mall_plain = pd.read_csv(path_mall_plain)
df_mall_spend = pd.read_csv(path_mall_spend)

df_mall_plain[W_COL] = 0  # ohne Spendentafel
df_mall_spend[W_COL] = 1  # mit Spendentafel

mall_dm = pd.concat([df_mall_plain, df_mall_spend], ignore_index=True)

# BeachBar: Codes ggf. anpassen, falls deine Ordner anders heißen
path_bb_plain = make_path("BB_T_D_M")
path_bb_spend = make_path("BB_T_D_SM")

df_bb_plain = pd.read_csv(path_bb_plain)
df_bb_spend = pd.read_csv(path_bb_spend)

df_bb_plain[W_COL] = 0
df_bb_spend[W_COL] = 1

beach_dm = pd.concat([df_bb_plain, df_bb_spend], ignore_index=True)

needed_cols = [
    X_COL, Y_COL, W_COL,
    "Social_Status_Old_1_4",
    "Social_Status_New_1_7",
]

mall_dm  = mall_dm[needed_cols].dropna().copy()
beach_dm = beach_dm[needed_cols].dropna().copy()

print(f"Sample size Mall (Donation Money with/without Spendentafel): {len(mall_dm)}")
print(f"Sample size BeachBar (Donation Money with/without Spendentafel): {len(beach_dm)}")

# ============================================================
# 4. Läufe: Mall + BeachBar, Status alt + neu
# ============================================================

# Mall – Status alt
res_mall_old = run_model5_moderation_and_export(
    data=mall_dm,
    x_col=X_COL,
    m_col="Social_Status_Old_1_4",
    y_col=Y_COL,
    w_col=W_COL,
    out_dir=OUTPUT_DIR_MALL,
    outfile_prefix="Mall_Model5_Status_Old",
    n_boot=N_BOOT,
    rng=rng,
)

# Mall – Status neu
res_mall_new = run_model5_moderation_and_export(
    data=mall_dm,
    x_col=X_COL,
    m_col="Social_Status_New_1_7",
    y_col=Y_COL,
    w_col=W_COL,
    out_dir=OUTPUT_DIR_MALL,
    outfile_prefix="Mall_Model5_Status_New",
    n_boot=N_BOOT,
    rng=rng,
)

# BeachBar – Status alt
res_bb_old = run_model5_moderation_and_export(
    data=beach_dm,
    x_col=X_COL,
    m_col="Social_Status_Old_1_4",
    y_col=Y_COL,
    w_col=W_COL,
    out_dir=OUTPUT_DIR_BB,
    outfile_prefix="BeachBar_Model5_Status_Old",
    n_boot=N_BOOT,
    rng=rng,
)

# BeachBar – Status neu
res_bb_new = run_model5_moderation_and_export(
    data=beach_dm,
    x_col=X_COL,
    m_col="Social_Status_New_1_7",
    y_col=Y_COL,
    w_col=W_COL,
    out_dir=OUTPUT_DIR_BB,
    outfile_prefix="BeachBar_Model5_Status_New",
    n_boot=N_BOOT,
    rng=rng,
)

print("\n✅ PROCESS-style Model 5 (c-path moderation by Spendentafel) completed for Mall and BeachBar.")


Sample size Mall (Donation Money with/without Spendentafel): 119
Sample size BeachBar (Donation Money with/without Spendentafel): 158

▶ Running c-path moderation (PROCESS-style Model 5): Mediator = Social_Status_Old_1_4, n = 119
Saved path table     → Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model5/Mall_Model5_Status_Old_paths.csv
Saved indirect table → Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model5/Mall_Model5_Status_Old_indirect.csv

APA-style summary
-----------------
Mediator: Social_Status_Old_1_4
n = 119

Total effect main (c_X): b = -15.6790, SE = 7.5501, t(117) = -2.0767, p = .0400, 95% CI [-30.6315, -0.7265]
Total effect interaction (c_XW): b = 0.0000, SE = 0.0000, t(117) = nan, p = n/a, 95% CI [0.0000, 0.0000]
Conditional total effects: W=0 (Private): c = -15.6790, W=1 (Public): c = -15.6790

Direct effect main (c'_X): b = -13.8320, SE = 7.6580, t(116) = -1.8062, p = .0735, 95% CI [-28.9996, 1.3357]
Direct effect interaction (c'_XW): b = 0.0000, SE =

## Model 8 -Moderation A and C' - Path

In [64]:

# ============================================================
# 0. Paths and settings
# ============================================================

INPUT_BASE_DIR = "../../../Analysis/Results/Per_Experiment_Recoded"

def make_path(exp_code: str, filename: str = "merged.csv") -> str:
    return os.path.join(INPUT_BASE_DIR, exp_code, filename)

OUTPUT_DIR_MALL = "Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model8"
OUTPUT_DIR_BEACHBAR = "Tables/BeachBar/Spendentafel/Donation_Money_Spendentafel/Model8"

os.makedirs(OUTPUT_DIR_MALL, exist_ok=True)
os.makedirs(OUTPUT_DIR_BEACHBAR, exist_ok=True)

X_COL = "Style_Glam0_Hippie1"           # Glam = 0, Hippie = 1
Y_COL = "DonationMoney"
W_COL = "Visibility_0_Private_1_Public" # 0 = Private, 1 = Public



# ============================================================
# 2. Core function: PROCESS Model 8–style moderated mediation
#    a-path and c'-path moderated by W
#
#    a-path:        M ~ X + W + XW
#    Y-model:       Y ~ X + W + XW + M
#    total effect:  Y ~ X + W + XW
#
#    Cond. indirects:
#      W = 0: a1 * b
#      W = 1: (a1 + a3) * b
# ============================================================

def run_process_model8_and_export(
    data: pd.DataFrame,
    x_col: str,
    m_col: str,
    y_col: str,
    w_col: str,
    outfile_prefix: str,
    output_dir: str,
    n_boot: int = 5000,
    seed: int = 42,
):
    rng = np.random.default_rng(seed)

    # Keep only needed cols
    sub = data[[x_col, m_col, y_col, w_col]].dropna().copy()
    n = len(sub)

    # interaction X*W
    sub["XW"] = sub[x_col] * sub[w_col]

    print(f"\n▶ Running PROCESS-style Model 8: Mediator = {m_col}, n = {n}")

    # -------------------------
    # a-path: M ~ X + W + XW
    # -------------------------
    Xa = sm.add_constant(sub[[x_col, w_col, "XW"]])
    model_a = sm.OLS(sub[m_col], Xa).fit()

    a1 = model_a.params[x_col]      # X
    a2 = model_a.params[w_col]      # W
    a3 = model_a.params["XW"]       # interaction

    a1_se = model_a.bse[x_col]
    a2_se = model_a.bse[w_col]
    a3_se = model_a.bse["XW"]

    a1_t = model_a.tvalues[x_col]
    a2_t = model_a.tvalues[w_col]
    a3_t = model_a.tvalues["XW"]

    a1_p = model_a.pvalues[x_col]
    a2_p = model_a.pvalues[w_col]
    a3_p = model_a.pvalues["XW"]

    a_ci = model_a.conf_int().round(4)
    a1_ll, a1_ul = a_ci.loc[x_col]
    a2_ll, a2_ul = a_ci.loc[w_col]
    a3_ll, a3_ul = a_ci.loc["XW"]

    # -------------------------
    # Y-model: Y ~ X + W + XW + M  (b-path and moderated c')
    # -------------------------
    Xy = sm.add_constant(sub[[x_col, w_col, "XW", m_col]])
    model_y = sm.OLS(sub[y_col], Xy).fit()

    # b-path: M -> Y
    b = model_y.params[m_col]
    b_se = model_y.bse[m_col]
    b_t = model_y.tvalues[m_col]
    b_p = model_y.pvalues[m_col]
    b_ll, b_ul = model_y.conf_int().loc[m_col].round(4)

    # d-path: W -> Y | X,M,XW
    d = model_y.params[w_col]
    d_se = model_y.bse[w_col]
    d_t = model_y.tvalues[w_col]
    d_p = model_y.pvalues[w_col]
    d_ll, d_ul = model_y.conf_int().loc[w_col].round(4)

    # c' main effect: X -> Y | W, XW, M
    c_main = model_y.params[x_col]
    c_main_se = model_y.bse[x_col]
    c_main_t = model_y.tvalues[x_col]
    c_main_p = model_y.pvalues[x_col]
    c_main_ll, c_main_ul = model_y.conf_int().loc[x_col].round(4)

    # c' interaction: XW -> Y
    c_int = model_y.params["XW"]
    c_int_se = model_y.bse["XW"]
    c_int_t = model_y.tvalues["XW"]
    c_int_p = model_y.pvalues["XW"]
    c_int_ll, c_int_ul = model_y.conf_int().loc["XW"].round(4)

    # conditional direct effects:
    cprime_private = c_main               # W=0
    cprime_public = c_main + c_int        # W=1

    # -------------------------
    # Total effect: Y ~ X + W + XW  (moderated c)
    # -------------------------
    Xt = sm.add_constant(sub[[x_col, w_col, "XW"]])
    model_c = sm.OLS(sub[y_col], Xt).fit()

    c_tot_main = model_c.params[x_col]
    c_tot_se_main = model_c.bse[x_col]
    c_tot_t_main = model_c.tvalues[x_col]
    c_tot_p_main = model_c.pvalues[x_col]
    c_tot_ll_main, c_tot_ul_main = model_c.conf_int().loc[x_col].round(4)

    c_tot_int = model_c.params["XW"]
    c_tot_se_int = model_c.bse["XW"]
    c_tot_t_int = model_c.tvalues["XW"]
    c_tot_p_int = model_c.pvalues["XW"]
    c_tot_ll_int, c_tot_ul_int = model_c.conf_int().loc["XW"].round(4)

    # conditional total effects:
    c_tot_private = c_tot_main            # W=0
    c_tot_public = c_tot_main + c_tot_int # W=1

    # -------------------------
    # Conditional indirect effects
    # -------------------------
    indirect_0 = a1 * b               # W=0
    indirect_1 = (a1 + a3) * b        # W=1

    # Bootstrap conditional indirects
    boot_0 = []
    boot_1 = []

    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        boot = sub.iloc[idx].copy()
        boot["XW"] = boot[x_col] * boot[w_col]

        # a-path bootstrap
        Xa_b = sm.add_constant(boot[[x_col, w_col, "XW"]])
        model_a_b = sm.OLS(boot[m_col], Xa_b).fit()
        a1_b = model_a_b.params[x_col]
        a3_b = model_a_b.params["XW"]

        # Y-model bootstrap
        Xy_b = sm.add_constant(boot[[x_col, w_col, "XW", m_col]])
        model_y_b = sm.OLS(boot[y_col], Xy_b).fit()
        b_b = model_y_b.params[m_col]

        boot_0.append(a1_b * b_b)
        boot_1.append((a1_b + a3_b) * b_b)

    boot_0 = np.array(boot_0)
    boot_1 = np.array(boot_1)

    se0 = float(boot_0.std())
    se1 = float(boot_1.std())

    ci0_low, ci0_high = np.percentile(boot_0, [2.5, 97.5]).round(4)
    ci1_low, ci1_high = np.percentile(boot_1, [2.5, 97.5]).round(4)

    sig0 = "Significant" if (ci0_low > 0 or ci0_high < 0) else "Not significant"
    sig1 = "Significant" if (ci1_low > 0 or ci1_high < 0) else "Not significant"

    # ========================================================
    # Build tables
    # ========================================================

    results_paths = pd.DataFrame({
        "Path": [
            "Total Effect main (c_X)",
            "Total Effect interaction (c_XW)",
            "Direct Effect main (c'_X)",
            "Direct Effect interaction (c'_XW)",
            "Path a1 (X → M)",
            "Path a2 (W → M)",
            "Path a3 (X×W → M)",
            "Path b (M → Y | X,W,XW)",
            "Path d (W → Y | X,M,XW)",
        ],
        "Coefficient (B)": [
            c_tot_main, c_tot_int,
            c_main, c_int,
            a1, a2, a3,
            b, d
        ],
        "SE": [
            c_tot_se_main, c_tot_se_int,
            c_main_se, c_int_se,
            a1_se, a2_se, a3_se,
            b_se, d_se
        ],
        "t-value": [
            c_tot_t_main, c_tot_t_int,
            c_main_t, c_int_t,
            a1_t, a2_t, a3_t,
            b_t, d_t
        ],
        "p-value": [
            c_tot_p_main, c_tot_p_int,
            c_main_p, c_int_p,
            a1_p, a2_p, a3_p,
            b_p, d_p
        ],
        "95% CI (LL)": [
            c_tot_ll_main, c_tot_ll_int,
            c_main_ll, c_int_ll,
            a1_ll, a2_ll, a3_ll,
            b_ll, d_ll
        ],
        "95% CI (UL)": [
            c_tot_ul_main, c_tot_ul_int,
            c_main_ul, c_int_ul,
            a1_ul, a2_ul, a3_ul,
            b_ul, d_ul
        ],
    })

    results_paths["Significance"] = results_paths["p-value"].apply(sig_label)

    results_indirect = pd.DataFrame({
        "Moderator_Value": [0, 1],
        "Moderator_Label": ["Private", "Public"],
        "Effect (a-path moderated)": [indirect_0, indirect_1],
        "Boot SE": [se0, se1],
        "BootLLCI": [ci0_low, ci1_low],
        "BootULCI": [ci0_high, ci1_high],
        "Significance": [sig0, sig1],
    })

    os.makedirs(output_dir, exist_ok=True)
    path_outfile = os.path.join(output_dir, f"{outfile_prefix}_paths.csv")
    indirect_outfile = os.path.join(output_dir, f"{outfile_prefix}_indirect.csv")

    results_paths.to_csv(path_outfile, float_format="%.4f", index=False)
    results_indirect.to_csv(indirect_outfile, float_format="%.4f", index=False)

    print(f"Saved path table     → {path_outfile}")
    print(f"Saved indirect table → {indirect_outfile}")

    # ========================================================
    # Console output (APA-style)
    # ========================================================

    print("\nAPA-style summary")
    print("-----------------")
    print(f"Mediator: {m_col}")
    print(f"n = {n}")

    # Total effect (moderated)
    print(
        f"Total effect main (c_X): b = {c_tot_main:.4f}, SE = {c_tot_se_main:.4f}, "
        f"t({int(model_c.df_resid)}) = {c_tot_t_main:.4f}, p {format_p(c_tot_p_main)}, "
        f"95% CI [{c_tot_ll_main:.4f}, {c_tot_ul_main:.4f}]"
    )
    print(
        f"Total effect interaction (c_XW): b = {c_tot_int:.4f}, SE = {c_tot_se_int:.4f}, "
        f"t({int(model_c.df_resid)}) = {c_tot_t_int:.4f}, p {format_p(c_tot_p_int)}, "
        f"95% CI [{c_tot_ll_int:.4f}, {c_tot_ul_int:.4f}]"
    )
    print(
        f"Conditional total effects: "
        f"W=0 (Private): c = {c_tot_private:.4f}, "
        f"W=1 (Public): c = {c_tot_public:.4f}"
    )

    # Direct effects
    print(
        f"\nDirect effect main (c'_X): b = {c_main:.4f}, SE = {c_main_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {c_main_t:.4f}, p {format_p(c_main_p)}, "
        f"95% CI [{c_main_ll:.4f}, {c_main_ul:.4f}]"
    )
    print(
        f"Direct effect interaction (c'_XW): b = {c_int:.4f}, SE = {c_int_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {c_int_t:.4f}, p {format_p(c_int_p)}, "
        f"95% CI [{c_int_ll:.4f}, {c_int_ul:.4f}]"
    )
    print(
        f"Conditional direct effects (c'): "
        f"W=0 (Private): c' = {cprime_private:.4f}, "
        f"W=1 (Public): c' = {cprime_public:.4f}"
    )

    # a-paths
    print(
        f"\nPath a1 (X → M): b = {a1:.4f}, SE = {a1_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a1_t:.4f}, p {format_p(a1_p)}, "
        f"95% CI [{a1_ll:.4f}, {a1_ul:.4f}]"
    )
    print(
        f"Path a2 (W → M): b = {a2:.4f}, SE = {a2_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a2_t:.4f}, p {format_p(a2_p)}, "
        f"95% CI [{a2_ll:.4f}, {a2_ul:.4f}]"
    )
    print(
        f"Path a3 (X×W → M): b = {a3:.4f}, SE = {a3_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a3_t:.4f}, p {format_p(a3_p)}, "
        f"95% CI [{a3_ll:.4f}, {a3_ul:.4f}]"
    )

    # b, d
    print(
        f"Path b (M → Y | X,W,XW): b = {b:.4f}, SE = {b_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b_t:.4f}, p {format_p(b_p)}, "
        f"95% CI [{b_ll:.4f}, {b_ul:.4f}]"
    )
    print(
        f"Path d (W → Y | X,M,XW): b = {d:.4f}, SE = {d_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {d_t:.4f}, p {format_p(d_p)}, "
        f"95% CI [{d_ll:.4f}, {d_ul:.4f}]"
    )

    # conditional indirects
    print("\nConditional indirect effects (moderated mediation via a-path):")
    print(
        f"- Visibility = 0 (Private): "
        f"Effect = {indirect_0:.4f}, Boot SE = {se0:.4f}, "
        f"95% CI [{ci0_low:.4f}, {ci0_high:.4f}] → {sig0}"
    )
    print(
        f"- Visibility = 1 (Public):  "
        f"Effect = {indirect_1:.4f}, Boot SE = {se1:.4f}, "
        f"95% CI [{ci1_low:.4f}, {ci1_high:.4f}] → {sig1}"
    )

    return results_paths, results_indirect


# ============================================================
# 3. Load Mall and BeachBar data (M_D_M/M_D_S_M, BB_T_D_M/BB_T_D_SM)
# ============================================================

# Mall
path_mall_plain = make_path("M_D_M")
path_mall_spend = make_path("M_D_S_M")

df_mall_plain = pd.read_csv(path_mall_plain)
df_mall_spend = pd.read_csv(path_mall_spend)

df_mall_plain[W_COL] = 0  # ohne Spendentafel = Private
df_mall_spend[W_COL] = 1  # mit Spendentafel = Public

mall_dm = pd.concat([df_mall_plain, df_mall_spend], ignore_index=True)

# BeachBar
path_bb_plain = make_path("BB_T_D_M")
path_bb_spend = make_path("BB_T_D_SM")

df_bb_plain = pd.read_csv(path_bb_plain)
df_bb_spend = pd.read_csv(path_bb_spend)

df_bb_plain[W_COL] = 0
df_bb_spend[W_COL] = 1

beach_dm = pd.concat([df_bb_plain, df_bb_spend], ignore_index=True)

needed_cols = [
    X_COL, Y_COL, W_COL,
    "Social_Status_Old_1_4",
    "Social_Status_New_1_7",
]

mall_dm = mall_dm[needed_cols].dropna().copy()
beach_dm = beach_dm[needed_cols].dropna().copy()

print(
    "Sample size Mall (Donation Money with/without Spendentafel):",
    len(mall_dm)
)
print(
    "Sample size BeachBar (Donation Money with/without Spendentafel):",
    len(beach_dm)
)

# make sure X and W are numeric
mall_dm[X_COL] = pd.to_numeric(mall_dm[X_COL], errors="coerce")
mall_dm[W_COL] = pd.to_numeric(mall_dm[W_COL], errors="coerce")
beach_dm[X_COL] = pd.to_numeric(beach_dm[X_COL], errors="coerce")
beach_dm[W_COL] = pd.to_numeric(beach_dm[W_COL], errors="coerce")

mall_dm = mall_dm.dropna(subset=[X_COL, W_COL])
beach_dm = beach_dm.dropna(subset=[X_COL, W_COL])

# ============================================================
# 4. Run for OLD and NEW status scales (Mall + BeachBar)
# ============================================================

# Mall – Status old
run_process_model8_and_export(
    data=mall_dm,
    x_col=X_COL,
    m_col="Social_Status_Old_1_4",
    y_col=Y_COL,
    w_col=W_COL,
    outfile_prefix="Mall_Model8_Status_Old",
    output_dir=OUTPUT_DIR_MALL,
)

# Mall – Status new
run_process_model8_and_export(
    data=mall_dm,
    x_col=X_COL,
    m_col="Social_Status_New_1_7",
    y_col=Y_COL,
    w_col=W_COL,
    outfile_prefix="Mall_Model8_Status_New",
    output_dir=OUTPUT_DIR_MALL,
)

# BeachBar – Status old
run_process_model8_and_export(
    data=beach_dm,
    x_col=X_COL,
    m_col="Social_Status_Old_1_4",
    y_col=Y_COL,
    w_col=W_COL,
    outfile_prefix="BeachBar_Model8_Status_Old",
    output_dir=OUTPUT_DIR_BEACHBAR,
)

# BeachBar – Status new
run_process_model8_and_export(
    data=beach_dm,
    x_col=X_COL,
    m_col="Social_Status_New_1_7",
    y_col=Y_COL,
    w_col=W_COL,
    outfile_prefix="BeachBar_Model8_Status_New",
    output_dir=OUTPUT_DIR_BEACHBAR,
)

print("\n✅ PROCESS-style Model 8 (a- and c'-path moderation by Spendentafel) completed for Mall and BeachBar.\n")


Sample size Mall (Donation Money with/without Spendentafel): 119
Sample size BeachBar (Donation Money with/without Spendentafel): 158

▶ Running PROCESS-style Model 8: Mediator = Social_Status_Old_1_4, n = 119
Saved path table     → Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model8/Mall_Model8_Status_Old_paths.csv
Saved indirect table → Tables/Mall/Spendentafel/Donation_Money_Spendentafel/Model8/Mall_Model8_Status_Old_indirect.csv

APA-style summary
-----------------
Mediator: Social_Status_Old_1_4
n = 119
Total effect main (c_X): b = -15.6790, SE = 7.5501, t(117) = -2.0767, p = .0400, 95% CI [-30.6315, -0.7265]
Total effect interaction (c_XW): b = 0.0000, SE = 0.0000, t(117) = nan, p = n/a, 95% CI [0.0000, 0.0000]
Conditional total effects: W=0 (Private): c = -15.6790, W=1 (Public): c = -15.6790

Direct effect main (c'_X): b = -13.8320, SE = 7.6580, t(116) = -1.8062, p = .0735, 95% CI [-28.9996, 1.3357]
Direct effect interaction (c'_XW): b = 0.0000, SE = 0.0000, t(116) = nan

## Awe

## Process Model 7 - A Path Moderation

In [65]:
import os
import pandas as pd

# ============================================================
# 0. Settings
# ============================================================

INPUT_PATH = "../../../Merging/Results/master_with_dv_means_recoded.csv"

BASE_OUT = "Tables/Mall/Awe"
OUTPUT_DIR_MODEL7 = os.path.join(BASE_OUT, "Model7")  # a-path moderated (PROCESS 7)

os.makedirs(OUTPUT_DIR_MODEL7, exist_ok=True)

# Columns
X_COL = "Style_Glam0_Hippie1"      # Glam = 0, Hippie = 1
Y_COL = "DonationMoney"
VIDEO_COL = "Awe_3cat"             # 0 = none, 1 = neutral, 2 = awe

STATUS_OLD = "Social_Status_Old_1_4"
STATUS_NEW = "Social_Status_New_1_7"

# Mall donation-money experiments (each corresponds to one Awe type)
MALL_EXPERIMENTS = ["M_D_M", "M_D_N_M", "M_D_A_M"]


# ============================================================
# 1. Load data and filter mall donation-money experiments
# ============================================================

data = pd.read_csv(INPUT_PATH)

mall_dm = data[data["Experiment_Type"].isin(MALL_EXPERIMENTS)].copy()

print("▶ Loaded master data")
print(f"Total n = {len(data)}")
print(f"Mall donation-money subset n = {len(mall_dm)}")
print("\nExperiment_Type in mall_dm:")
print(mall_dm["Experiment_Type"].value_counts(dropna=False))
print("\nAwe_3cat in mall_dm:")
print(mall_dm[VIDEO_COL].value_counts(dropna=False))


# ============================================================
# 2. Helper to prepare pairwise experiment subsets
# ============================================================

PAIR_DEFS = [
    # (pair_id, description, list_of_experiment_types)
    ("0_1", "NoVideo_vs_Neutral", ["M_D_M", "M_D_N_M"]),  # 0 vs 1
    ("1_2", "Neutral_vs_Awe",     ["M_D_N_M", "M_D_A_M"]),  # 1 vs 2
    ("0_2", "NoVideo_vs_Awe",     ["M_D_M", "M_D_A_M"]),    # 0 vs 2
]


def prepare_pair_subset(df: pd.DataFrame, exp_types, pair_id: str, pair_label: str):
    """
    Filter df by the given Experiment_Type list and show basic info.
    Moderator stays Awe_3cat (0, 1, 2).
    """
    sub = df[df["Experiment_Type"].isin(exp_types)].copy()

    print(f"\n▶ Preparing pair {pair_id}: {pair_label}")
    print(f"Included Experiment_Types: {exp_types}")
    print(f"n = {len(sub)}")

    print("Experiment_Type counts in this pair:")
    print(sub["Experiment_Type"].value_counts(dropna=False))

    print("Awe_3cat counts in this pair:")
    print(sub[VIDEO_COL].value_counts(dropna=False))

    return sub


# ============================================================
# 3. Run PROCESS Model 7 for each pair, both Status mediators
# ============================================================

# NOTE: Assumes `run_process_model7_and_export` is already defined
# exactly as in your previous code and available in this environment.

for pair_id, pair_label, exp_types in PAIR_DEFS:
    # 3.1 Prepare subset for this pair
    pair_df = prepare_pair_subset(mall_dm, exp_types, pair_id, pair_label)

    # Output directory for this pair
    pair_out_dir = os.path.join(OUTPUT_DIR_MODEL7, f"Pair_{pair_id}_{pair_label}")
    os.makedirs(pair_out_dir, exist_ok=True)

    # 3.2 Mediator: Social Status New (1–7)
    print(f"\n▶ Running PROCESS Model 7 for Status_New, pair {pair_id} ({pair_label})")
    run_process_model7_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_NEW,
        y_col=Y_COL,
        w_col=VIDEO_COL,  # Awe_3cat as moderator (0/1/2, but only 2 levels per pair)
        dir=pair_out_dir,
        outfile_prefix=f"StatusNew_{pair_label}_DonationMoney",
    )

    # 3.3 Mediator: Social Status Old (1–4)
    print(f"\n▶ Running PROCESS Model 7 for Status_Old, pair {pair_id} ({pair_label})")
    run_process_model7_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_OLD,
        y_col=Y_COL,
        w_col=VIDEO_COL,
        dir=pair_out_dir,
        outfile_prefix=f"StatusOld_{pair_label}_DonationMoney",
    )

print("\n✅ Finished PROCESS Model 7 analyses for DonationMoney × Status (Old + New) moderated by Awe (pairs of experiments).")


▶ Loaded master data
Total n = 1883
Mall donation-money subset n = 254

Experiment_Type in mall_dm:
Experiment_Type
M_D_M      119
M_D_A_M     72
M_D_N_M     63
Name: count, dtype: int64

Awe_3cat in mall_dm:
Awe_3cat
1.0    182
2.0     72
Name: count, dtype: int64

▶ Preparing pair 0_1: NoVideo_vs_Neutral
Included Experiment_Types: ['M_D_M', 'M_D_N_M']
n = 182
Experiment_Type counts in this pair:
Experiment_Type
M_D_M      119
M_D_N_M     63
Name: count, dtype: int64
Awe_3cat counts in this pair:
Awe_3cat
1.0    182
Name: count, dtype: int64

▶ Running PROCESS Model 7 for Status_New, pair 0_1 (NoVideo_vs_Neutral)

▶ Running PROCESS Model 7: Mediator = Social_Status_New_1_7, n = 119
Saved path table     → Tables/Mall/Awe/Model7/Pair_0_1_NoVideo_vs_Neutral/StatusNew_NoVideo_vs_Neutral_DonationMoney_paths.csv
Saved indirect table → Tables/Mall/Awe/Model7/Pair_0_1_NoVideo_vs_Neutral/StatusNew_NoVideo_vs_Neutral_DonationMoney_indirect.csv

APA-style summary
-----------------
Mediator: Soci

## Process Model 5 Moderation on the c Path 

In [66]:
import os
import numpy as np
import pandas as pd

# ============================================================
# 0. Settings
# ============================================================

INPUT_PATH = "../../../Merging/Results/master_with_dv_means_recoded.csv"

BASE_OUT = "Tables/Mall/Awe"
OUTPUT_DIR_MODEL5 = os.path.join(BASE_OUT, "Model5")  # c'-path moderated (PROCESS 5)

os.makedirs(OUTPUT_DIR_MODEL5, exist_ok=True)

# Columns
X_COL = "Style_Glam0_Hippie1"      # Glam = 0, Hippie = 1
Y_COL = "DonationMoney"
VIDEO_COL = "Awe_3cat"             # 0 = none, 1 = neutral, 2 = awe

STATUS_OLD = "Social_Status_Old_1_4"
STATUS_NEW = "Social_Status_New_1_7"

# Mall donation-money experiments (each corresponds to one Awe type)
MALL_EXPERIMENTS = ["M_D_M", "M_D_N_M", "M_D_A_M"]

# Bootstrap settings for Model 5
N_BOOT = 5000
rng = np.random.default_rng(42)


# ============================================================
# 1. Load data and filter mall donation-money experiments
# ============================================================

data = pd.read_csv(INPUT_PATH)

mall_dm = data[data["Experiment_Type"].isin(MALL_EXPERIMENTS)].copy()

# Ensure Awe_3cat is consistent with Experiment_Type (0 = none, 1 = neutral, 2 = awe)
exp_to_awe = {
    "M_D_M": 0,  # no video
    "M_D_N_M": 1,  # neutral
    "M_D_A_M": 2,  # awe
}
mall_dm[VIDEO_COL] = mall_dm["Experiment_Type"].map(exp_to_awe)

print("▶ Loaded master data")
print(f"Total n = {len(data)}")
print(f"Mall donation-money subset n = {len(mall_dm)}")
print("\nExperiment_Type in mall_dm:")
print(mall_dm["Experiment_Type"].value_counts(dropna=False))
print("\nAwe_3cat in mall_dm after recode:")
print(mall_dm[VIDEO_COL].value_counts(dropna=False))


# ============================================================
# 2. Helper to prepare pairwise experiment subsets
# ============================================================

PAIR_DEFS = [
    # (pair_id, description, list_of_experiment_types)
    ("0_1", "NoVideo_vs_Neutral", ["M_D_M", "M_D_N_M"]),   # 0 vs 1
    ("1_2", "Neutral_vs_Awe",     ["M_D_N_M", "M_D_A_M"]), # 1 vs 2
    ("0_2", "NoVideo_vs_Awe",     ["M_D_M", "M_D_A_M"]),   # 0 vs 2
]


def prepare_pair_subset(df: pd.DataFrame, exp_types, pair_id: str, pair_label: str):
    """
    Filter df by the given Experiment_Type list and show basic info.
    Moderator stays Awe_3cat (0, 1, 2), but each pair only uses 2 levels.
    """
    sub = df[df["Experiment_Type"].isin(exp_types)].copy()

    print(f"\n▶ Preparing pair {pair_id}: {pair_label}")
    print(f"Included Experiment_Types: {exp_types}")
    print(f"n = {len(sub)}")

    print("Experiment_Type counts in this pair:")
    print(sub["Experiment_Type"].value_counts(dropna=False))

    print("Awe_3cat counts in this pair:")
    print(sub[VIDEO_COL].value_counts(dropna=False))

    return sub


# ============================================================
# 3. Run PROCESS Model 5 for each pair, both Status mediators
# ============================================================

# NOTE: Assumes `run_model5_moderation_and_export` is already defined
# in the environment with signature:
# run_model5_moderation_and_export(
#     data, x_col, m_col, y_col, w_col, out_dir, outfile_prefix, n_boot, rng
# )

for pair_id, pair_label, exp_types in PAIR_DEFS:
    # 3.1 Prepare subset for this pair
    pair_df = prepare_pair_subset(mall_dm, exp_types, pair_id, pair_label)

    # Output directory for this pair
    pair_out_dir = os.path.join(OUTPUT_DIR_MODEL5, f"Pair_{pair_id}_{pair_label}")
    os.makedirs(pair_out_dir, exist_ok=True)

    # 3.2 Mediator: Social Status New (1–7)
    print(f"\n▶ Running PROCESS Model 5 for Status_New, pair {pair_id} ({pair_label})")
    run_model5_moderation_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_NEW,
        y_col=Y_COL,
        w_col=VIDEO_COL,  # Awe_3cat as moderator (0/1/2, but only 2 levels per pair)
        out_dir=pair_out_dir,
        outfile_prefix=f"StatusNew_{pair_label}_DonationMoney",
        n_boot=N_BOOT,
        rng=rng,
    )

    # 3.3 Mediator: Social Status Old (1–4)
    print(f"\n▶ Running PROCESS Model 5 for Status_Old, pair {pair_id} ({pair_label})")
    run_model5_moderation_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_OLD,
        y_col=Y_COL,
        w_col=VIDEO_COL,
        out_dir=pair_out_dir,
        outfile_prefix=f"StatusOld_{pair_label}_DonationMoney",
        n_boot=N_BOOT,
        rng=rng,
    )

print("\n✅ Finished PROCESS Model 5 analyses for DonationMoney × Status (Old + New) moderated by Awe (pairs of experiments).")


▶ Loaded master data
Total n = 1883
Mall donation-money subset n = 254

Experiment_Type in mall_dm:
Experiment_Type
M_D_M      119
M_D_A_M     72
M_D_N_M     63
Name: count, dtype: int64

Awe_3cat in mall_dm after recode:
Awe_3cat
0    119
2     72
1     63
Name: count, dtype: int64

▶ Preparing pair 0_1: NoVideo_vs_Neutral
Included Experiment_Types: ['M_D_M', 'M_D_N_M']
n = 182
Experiment_Type counts in this pair:
Experiment_Type
M_D_M      119
M_D_N_M     63
Name: count, dtype: int64
Awe_3cat counts in this pair:
Awe_3cat
0    119
1     63
Name: count, dtype: int64

▶ Running PROCESS Model 5 for Status_New, pair 0_1 (NoVideo_vs_Neutral)

▶ Running c-path moderation (PROCESS-style Model 5): Mediator = Social_Status_New_1_7, n = 119
Saved path table     → Tables/Mall/Awe/Model5/Pair_0_1_NoVideo_vs_Neutral/StatusNew_NoVideo_vs_Neutral_DonationMoney_paths.csv
Saved indirect table → Tables/Mall/Awe/Model5/Pair_0_1_NoVideo_vs_Neutral/StatusNew_NoVideo_vs_Neutral_DonationMoney_indirect.csv


## Process Model 8 Moderation on a and c path 

In [67]:
import os
import numpy as np
import pandas as pd

# ============================================================
# 0. Settings
# ============================================================

INPUT_PATH = "../../../Merging/Results/master_with_dv_means_recoded.csv"

BASE_OUT = "Tables/Mall/Donation_Money_Awe"
OUTPUT_DIR_MODEL8 = os.path.join(BASE_OUT, "Model8")  # a- and c-path moderated (PROCESS 8)

os.makedirs(OUTPUT_DIR_MODEL8, exist_ok=True)

# Columns
X_COL = "Style_Glam0_Hippie1"      # Glam = 0, Hippie = 1
Y_COL = "DonationMoney"
VIDEO_COL = "Awe_3cat"             # 0 = none, 1 = neutral, 2 = awe

STATUS_OLD = "Social_Status_Old_1_4"
STATUS_NEW = "Social_Status_New_1_7"

# Mall donation-money experiments (each corresponds to one Awe type)
MALL_EXPERIMENTS = ["M_D_M", "M_D_N_M", "M_D_A_M"]

# Bootstrap settings for Model 8
N_BOOT = 5000
rng = np.random.default_rng(42)


# ============================================================
# 1. Load data and filter mall donation-money experiments
# ============================================================

data = pd.read_csv(INPUT_PATH)

mall_dm = data[data["Experiment_Type"].isin(MALL_EXPERIMENTS)].copy()

# Ensure Awe_3cat is consistent with Experiment_Type (0 = none, 1 = neutral, 2 = awe)
exp_to_awe = {
    "M_D_M": 0,  # no video
    "M_D_N_M": 1,  # neutral
    "M_D_A_M": 2,  # awe
}
mall_dm[VIDEO_COL] = mall_dm["Experiment_Type"].map(exp_to_awe)

print("▶ Loaded master data")
print(f"Total n = {len(data)}")
print(f"Mall donation-money subset n = {len(mall_dm)}")
print("\nExperiment_Type in mall_dm:")
print(mall_dm["Experiment_Type"].value_counts(dropna=False))
print("\nAwe_3cat in mall_dm after recode:")
print(mall_dm[VIDEO_COL].value_counts(dropna=False))


# ============================================================
# 2. Helper to prepare pairwise experiment subsets
# ============================================================

PAIR_DEFS = [
    # (pair_id, description, list_of_experiment_types)
    ("0_1", "NoVideo_vs_Neutral", ["M_D_M", "M_D_N_M"]),   # 0 vs 1
    ("1_2", "Neutral_vs_Awe",     ["M_D_N_M", "M_D_A_M"]), # 1 vs 2
    ("0_2", "NoVideo_vs_Awe",     ["M_D_M", "M_D_A_M"]),   # 0 vs 2
]


def prepare_pair_subset(df: pd.DataFrame, exp_types, pair_id: str, pair_label: str):
    """
    Filter df by the given Experiment_Type list and show basic info.
    Moderator stays Awe_3cat (0, 1, 2), but each pair only uses 2 levels.
    """
    sub = df[df["Experiment_Type"].isin(exp_types)].copy()

    print(f"\n▶ Preparing pair {pair_id}: {pair_label}")
    print(f"Included Experiment_Types: {exp_types}")
    print(f"n = {len(sub)}")

    print("Experiment_Type counts in this pair:")
    print(sub["Experiment_Type"].value_counts(dropna=False))

    print("Awe_3cat counts in this pair:")
    print(sub[VIDEO_COL].value_counts(dropna=False))

    return sub


# ============================================================
# 3. Run PROCESS Model 8 for each pair, both Status mediators
# ============================================================

# NOTE: Assumes `run_model8_moderation_and_export` is already defined
# with signature:
# run_model8_moderation_and_export(
#     data, x_col, m_col, y_col, w_col, out_dir, outfile_prefix, n_boot, rng
# )

for pair_id, pair_label, exp_types in PAIR_DEFS:
    # 3.1 Prepare subset for this pair
    pair_df = prepare_pair_subset(mall_dm, exp_types, pair_id, pair_label)

    # Output directory for this pair
    pair_out_dir = os.path.join(OUTPUT_DIR_MODEL8, f"Pair_{pair_id}_{pair_label}")
    os.makedirs(pair_out_dir, exist_ok=True)

    # 3.2 Mediator: Social Status New (1–7)
    print(f"\n▶ Running PROCESS Model 8 for Status_New, pair {pair_id} ({pair_label})")
    run_process_model8_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_NEW,
        y_col=Y_COL,
        w_col=VIDEO_COL,  # Awe_3cat as moderator (0/1/2, but only 2 levels per pair)
        output_dir=pair_out_dir,
        outfile_prefix=f"StatusNew_{pair_label}_DonationMoney",
        n_boot=N_BOOT
    )

    # 3.3 Mediator: Social Status Old (1–4)
    print(f"\n▶ Running PROCESS Model 8 for Status_Old, pair {pair_id} ({pair_label})")
    run_process_model8_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_OLD,
        y_col=Y_COL,
        w_col=VIDEO_COL,
        output_dir=pair_out_dir,
        outfile_prefix=f"StatusOld_{pair_label}_DonationMoney",
        n_boot=N_BOOT
    )

print("\n✅ Finished PROCESS Model 8 analyses for DonationMoney × Status (Old + New) moderated by Awe on a- and c-path (pairs of experiments).")


▶ Loaded master data
Total n = 1883
Mall donation-money subset n = 254

Experiment_Type in mall_dm:
Experiment_Type
M_D_M      119
M_D_A_M     72
M_D_N_M     63
Name: count, dtype: int64

Awe_3cat in mall_dm after recode:
Awe_3cat
0    119
2     72
1     63
Name: count, dtype: int64

▶ Preparing pair 0_1: NoVideo_vs_Neutral
Included Experiment_Types: ['M_D_M', 'M_D_N_M']
n = 182
Experiment_Type counts in this pair:
Experiment_Type
M_D_M      119
M_D_N_M     63
Name: count, dtype: int64
Awe_3cat counts in this pair:
Awe_3cat
0    119
1     63
Name: count, dtype: int64

▶ Running PROCESS Model 8 for Status_New, pair 0_1 (NoVideo_vs_Neutral)

▶ Running PROCESS-style Model 8: Mediator = Social_Status_New_1_7, n = 119
Saved path table     → Tables/Mall/Donation_Money_Awe/Model8/Pair_0_1_NoVideo_vs_Neutral/StatusNew_NoVideo_vs_Neutral_DonationMoney_paths.csv
Saved indirect table → Tables/Mall/Donation_Money_Awe/Model8/Pair_0_1_NoVideo_vs_Neutral/StatusNew_NoVideo_vs_Neutral_DonationMoney_ind

## Process Model 14  Moderation on b path

In [68]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

def run_process_model14_and_export(
    data: pd.DataFrame,
    x_col: str,
    m_col: str,
    y_col: str,
    w_col: str,
    dir: str,
    outfile_prefix: str,
    n_boot: int = 5000,
    seed: int = 42,
):
    """
    PROCESS-style Model 14:
    - a-path unmoderated: M ~ X + W
    - b-path moderated:   Y ~ X + M + W + M*W
    - Total effect:       Y ~ X + W
    - Conditional indirect effects via moderated b-path:
          Indirect(w) = a * (b1 + b3*w)
    """

    os.makedirs(dir, exist_ok=True)
    rng = np.random.default_rng(seed)

    sub = data[[x_col, m_col, y_col, w_col]].dropna().copy()
    n = len(sub)
    if n == 0:
        print(
            f"\n⚠ Skipping {outfile_prefix}: "
            f"no complete cases for [{x_col}, {m_col}, {y_col}, {w_col}] after dropna."
        )
        return None, None

    # ensure we have at least two W levels
    uniq_w = sorted(sub[w_col].unique())
    if len(uniq_w) < 2:
        print(
            f"\n⚠ Skipping {outfile_prefix}: moderator {w_col} "
            f"has < 2 distinct values in this subset."
        )
        return None, None

    # We will compute conditional indirect effects at the lowest and highest W
    w_low, w_high = uniq_w[0], uniq_w[-1]

    print(f"\n▶ Running PROCESS Model 14: Mediator = {m_col}, n = {n}")
    print(f"  Moderator {w_col}: using W_low = {w_low}, W_high = {w_high}")

    # ----------------------------------------------------
    # a-path model (unmoderated): M ~ X + W
    # ----------------------------------------------------
    Xa = sm.add_constant(sub[[x_col, w_col]])
    model_a = sm.OLS(sub[m_col], Xa).fit()

    a1 = round(model_a.params[x_col], 4)   # X → M
    a2 = round(model_a.params[w_col], 4)   # W → M

    a1_se = round(model_a.bse[x_col], 4)
    a2_se = round(model_a.bse[w_col], 4)

    a1_t = round(model_a.tvalues[x_col], 4)
    a2_t = round(model_a.tvalues[w_col], 4)

    a1_p = round(model_a.pvalues[x_col], 4)
    a2_p = round(model_a.pvalues[w_col], 4)

    a1_ll, a1_ul = model_a.conf_int().loc[x_col].round(4)
    a2_ll, a2_ul = model_a.conf_int().loc[w_col].round(4)

    # ----------------------------------------------------
    # Y model (b-path moderated): Y ~ X + M + W + M*W
    # ----------------------------------------------------
    sub["MW"] = sub[m_col] * sub[w_col]
    Xy = sm.add_constant(sub[[x_col, m_col, w_col, "MW"]])
    model_y = sm.OLS(sub[y_col], Xy).fit()

    b1 = round(model_y.params[m_col], 4)      # M → Y
    b2 = round(model_y.params[w_col], 4)      # W → Y
    b3 = round(model_y.params["MW"], 4)       # M×W → Y
    cprime = round(model_y.params[x_col], 4)  # X → Y | M,W,MW

    b1_se = round(model_y.bse[m_col], 4)
    b2_se = round(model_y.bse[w_col], 4)
    b3_se = round(model_y.bse["MW"], 4)
    cprime_se = round(model_y.bse[x_col], 4)

    b1_t = round(model_y.tvalues[m_col], 4)
    b2_t = round(model_y.tvalues[w_col], 4)
    b3_t = round(model_y.tvalues["MW"], 4)
    cprime_t = round(model_y.tvalues[x_col], 4)

    b1_p = round(model_y.pvalues[m_col], 4)
    b2_p = round(model_y.pvalues[w_col], 4)
    b3_p = round(model_y.pvalues["MW"], 4)
    cprime_p = round(model_y.pvalues[x_col], 4)

    b1_ll, b1_ul = model_y.conf_int().loc[m_col].round(4)
    b2_ll, b2_ul = model_y.conf_int().loc[w_col].round(4)
    b3_ll, b3_ul = model_y.conf_int().loc["MW"].round(4)
    cprime_ll, cprime_ul = model_y.conf_int().loc[x_col].round(4)

    # ----------------------------------------------------
    # Total effect c: Y ~ X + W
    # ----------------------------------------------------
    Xt = sm.add_constant(sub[[x_col, w_col]])
    model_c = sm.OLS(sub[y_col], Xt).fit()

    c = round(model_c.params[x_col], 4)
    c_se = round(model_c.bse[x_col], 4)
    c_t = round(model_c.tvalues[x_col], 4)
    c_p = round(model_c.pvalues[x_col], 4)
    c_ll, c_ul = model_c.conf_int().loc[x_col].round(4)

    # ----------------------------------------------------
    # Conditional indirect effects (b-path moderated)
    # Indirect(w) = a * (b1 + b3 * w)
    # ----------------------------------------------------
    indirect_low = round(a1 * (b1 + b3 * w_low), 4)
    indirect_high = round(a1 * (b1 + b3 * w_high), 4)

    # Bootstrap for both conditional indirects
    boot_low = []
    boot_high = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        boot = sub.iloc[idx].copy()

        # a-model
        Xa_b = sm.add_constant(boot[[x_col, w_col]])
        a_b = sm.OLS(boot[m_col], Xa_b).fit()

        # y-model
        boot["MW_b"] = boot[m_col] * boot[w_col]
        Xy_b = sm.add_constant(boot[[x_col, m_col, w_col, "MW_b"]])
        y_b = sm.OLS(boot[y_col], Xy_b).fit()

        a1_b = a_b.params[x_col]
        b1_b = y_b.params[m_col]
        b3_b = y_b.params["MW_b"]

        boot_low.append(a1_b * (b1_b + b3_b * w_low))
        boot_high.append(a1_b * (b1_b + b3_b * w_high))

    boot_low = np.array(boot_low)
    boot_high = np.array(boot_high)

    ci_low_lo, ci_low_hi = np.percentile(boot_low, [2.5, 97.5]).round(4)
    ci_high_lo, ci_high_hi = np.percentile(boot_high, [2.5, 97.5]).round(4)

    se_low = round(boot_low.std(), 4)
    se_high = round(boot_high.std(), 4)

    sig_low = "Significant" if (ci_low_lo > 0 or ci_low_hi < 0) else "Not significant"
    sig_high = "Significant" if (ci_high_lo > 0 or ci_high_hi < 0) else "Not significant"

    # ========================================================
    # Build tables
    # ========================================================

    results_paths = pd.DataFrame({
        "Path": [
            "Total Effect (c)",
            "Direct Effect (c')",
            "Path a1 (X → M)",
            "Path a2 (W → M)",
            "Path b1 (M → Y | X,W,MW)",
            "Path b2 (W → Y | X,M,MW)",
            "Path b3 (M×W → Y)",
        ],
        "Coefficient (B)": [c, cprime, a1, a2, b1, b2, b3],
        "SE": [c_se, cprime_se, a1_se, a2_se, b1_se, b2_se, b3_se],
        "t-value": [c_t, cprime_t, a1_t, a2_t, b1_t, b2_t, b3_t],
        "p-value": [c_p, cprime_p, a1_p, a2_p, b1_p, b2_p, b3_p],
        "95% CI (LL)": [c_ll, cprime_ll, a1_ll, a2_ll, b1_ll, b2_ll, b3_ll],
        "95% CI (UL)": [c_ul, cprime_ul, a1_ul, a2_ul, b1_ul, b2_ul, b3_ul],
    })
    results_paths["Significance"] = results_paths["p-value"].apply(sig_label)

    results_indirect = pd.DataFrame({
        "Moderator_Value": [w_low, w_high],
        "Moderator_Label": [f"{w_col}={w_low}", f"{w_col}={w_high}"],
        "Effect (b-path moderated)": [indirect_low, indirect_high],
        "Boot SE": [se_low, se_high],
        "BootLLCI": [ci_low_lo, ci_high_lo],
        "BootULCI": [ci_low_hi, ci_high_hi],
        "Significance": [sig_low, sig_high],
    })

    path_outfile = os.path.join(dir, f"{outfile_prefix}_paths.csv")
    indirect_outfile = os.path.join(dir, f"{outfile_prefix}_indirect.csv")

    results_paths.to_csv(path_outfile, float_format="%.4f", index=False)
    results_indirect.to_csv(indirect_outfile, float_format="%.4f", index=False)

    print(f"Saved path table     → {path_outfile}")
    print(f"Saved indirect table → {indirect_outfile}")

    # ========================================================
    # Console output (APA-style)
    # ========================================================
    print("\nAPA-style summary")
    print("-----------------")
    print(f"Mediator: {m_col}")
    print(f"n = {n}")

    print(
        f"Total effect (c): b = {c:.4f}, SE = {c_se:.4f}, "
        f"t({int(model_c.df_resid)}) = {c_t:.4f}, p {format_p(c_p)}, "
        f"95% CI [{c_ll:.4f}, {c_ul:.4f}]"
    )

    print(
        f"Direct effect (c'): b = {cprime:.4f}, SE = {cprime_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {cprime_t:.4f}, p {format_p(cprime_p)}, "
        f"95% CI [{cprime_ll:.4f}, {cprime_ul:.4f}]"
    )

    print(
        f"Path a1 (X → M): b = {a1:.4f}, SE = {a1_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a1_t:.4f}, p {format_p(a1_p)}, "
        f"95% CI [{a1_ll:.4f}, {a1_ul:.4f}]"
    )
    print(
        f"Path a2 (W → M): b = {a2:.4f}, SE = {a2_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a2_t:.4f}, p {format_p(a2_p)}, "
        f"95% CI [{a2_ll:.4f}, {a2_ul:.4f}]"
    )

    print(
        f"Path b1 (M → Y | X,W,MW): b = {b1:.4f}, SE = {b1_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b1_t:.4f}, p {format_p(b1_p)}, "
        f"95% CI [{b1_ll:.4f}, {b1_ul:.4f}]"
    )
    print(
        f"Path b2 (W → Y | X,M,MW): b = {b2:.4f}, SE = {b2_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b2_t:.4f}, p {format_p(b2_p)}, "
        f"95% CI [{b2_ll:.4f}, {b2_ul:.4f}]"
    )
    print(
        f"Path b3 (M×W → Y): b = {b3:.4f}, SE = {b3_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b3_t:.4f}, p {format_p(b3_p)}, "
        f"95% CI [{b3_ll:.4f}, {b3_ul:.4f}]"
    )

    print("\nConditional indirect effects (b-path moderated):")
    print(
        f"- W = {w_low}: "
        f"Effect = {indirect_low:.4f}, Boot SE = {se_low:.4f}, "
        f"95% CI [{ci_low_lo:.4f}, {ci_low_hi:.4f}] → {sig_low}"
    )
    print(
        f"- W = {w_high}: "
        f"Effect = {indirect_high:.4f}, Boot SE = {se_high:.4f}, "
        f"95% CI [{ci_high_lo:.4f}, {ci_high_hi:.4f}] → {sig_high}"
    )

    return results_paths, results_indirect


## Process Model 15

In [ ]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

# If you already have these helpers, keep your versions and delete mine.
def sig_label(p):
    return "p < .05" if p < 0.05 else "n.s."

def format_p(p):
    if p < 0.001:
        return "< .001"
    return f"= {p:.3f}"


def run_process_model15_and_export(
    data: pd.DataFrame,
    x_col: str,
    m_col: str,
    y_col: str,
    w_col: str,
    out_dir: str,
    outfile_prefix: str,
    n_boot: int = 5000,
    seed: int = 42,
):
    """
    PROCESS-style Model 15:
      M = i1 + a1*X + a2*W + a3*XW + eM   (a-path moderated by W)
      Y = i2 + c'*X + b1*M + b2*W + b3*MW + eY   (b-path moderated by W)

    Total effect:
      Y = i3 + c*X + d*W + e

    Conditional indirect effect:
      Indirect(W) = (a1 + a3*W) * (b1 + b3*W)
    """

    os.makedirs(out_dir, exist_ok=True)

    rng = np.random.default_rng(seed)

    sub = data[[x_col, m_col, y_col, w_col]].dropna().copy()
    n = len(sub)
    if n == 0:
        print(
            f"\n⚠ Skipping {outfile_prefix}: "
            f"no complete cases for [{x_col}, {m_col}, {y_col}, {w_col}] after dropna."
        )
        return None, None

    uniq_w = sorted(sub[w_col].unique())
    if len(uniq_w) < 2:
        print(
            f"\n⚠ Skipping {outfile_prefix}: moderator {w_col} "
            f"has < 2 distinct values in this subset."
        )
        return None, None

    w_low, w_high = uniq_w[0], uniq_w[-1]

    print(f"\n▶ Running PROCESS Model 15: Mediator = {m_col}, n = {n}")
    print(f"  Moderator {w_col}: using W_low = {w_low}, W_high = {w_high}")

    # -------------------------
    # a-path model: M ~ X + W + XW
    # -------------------------
    sub["XW"] = sub[x_col] * sub[w_col]
    Xa = sm.add_constant(sub[[x_col, w_col, "XW"]])
    model_a = sm.OLS(sub[m_col], Xa).fit()

    a1 = round(model_a.params[x_col], 4)
    a2 = round(model_a.params[w_col], 4)
    a3 = round(model_a.params["XW"], 4)

    a1_se = round(model_a.bse[x_col], 4)
    a2_se = round(model_a.bse[w_col], 4)
    a3_se = round(model_a.bse["XW"], 4)

    a1_t = round(model_a.tvalues[x_col], 4)
    a2_t = round(model_a.tvalues[w_col], 4)
    a3_t = round(model_a.tvalues["XW"], 4)

    a1_p = round(model_a.pvalues[x_col], 4)
    a2_p = round(model_a.pvalues[w_col], 4)
    a3_p = round(model_a.pvalues["XW"], 4)

    a1_ll, a1_ul = model_a.conf_int().loc[x_col].round(4)
    a2_ll, a2_ul = model_a.conf_int().loc[w_col].round(4)
    a3_ll, a3_ul = model_a.conf_int().loc["XW"].round(4)

    # -------------------------
    # Y model: Y ~ X + M + W + MW  (b-path moderated)
    # -------------------------
    sub["MW"] = sub[m_col] * sub[w_col]
    Xy = sm.add_constant(sub[[x_col, m_col, w_col, "MW"]])
    model_y = sm.OLS(sub[y_col], Xy).fit()

    b1 = round(model_y.params[m_col], 4)
    b2 = round(model_y.params[w_col], 4)
    b3 = round(model_y.params["MW"], 4)
    cprime = round(model_y.params[x_col], 4)

    b1_se = round(model_y.bse[m_col], 4)
    b2_se = round(model_y.bse[w_col], 4)
    b3_se = round(model_y.bse["MW"], 4)
    cprime_se = round(model_y.bse[x_col], 4)

    b1_t = round(model_y.tvalues[m_col], 4)
    b2_t = round(model_y.tvalues[w_col], 4)
    b3_t = round(model_y.tvalues["MW"], 4)
    cprime_t = round(model_y.tvalues[x_col], 4)

    b1_p = round(model_y.pvalues[m_col], 4)
    b2_p = round(model_y.pvalues[w_col], 4)
    b3_p = round(model_y.pvalues["MW"], 4)
    cprime_p = round(model_y.pvalues[x_col], 4)

    b1_ll, b1_ul = model_y.conf_int().loc[m_col].round(4)
    b2_ll, b2_ul = model_y.conf_int().loc[w_col].round(4)
    b3_ll, b3_ul = model_y.conf_int().loc["MW"].round(4)
    cprime_ll, cprime_ul = model_y.conf_int().loc[x_col].round(4)

    # -------------------------
    # Total effect: Y ~ X + W
    # -------------------------
    Xt = sm.add_constant(sub[[x_col, w_col]])
    model_c = sm.OLS(sub[y_col], Xt).fit()

    c = round(model_c.params[x_col], 4)
    c_se = round(model_c.bse[x_col], 4)
    c_t = round(model_c.tvalues[x_col], 4)
    c_p = round(model_c.pvalues[x_col], 4)
    c_ll, c_ul = model_c.conf_int().loc[x_col].round(4)

    # -------------------------
    # Conditional indirect effects (a and b moderated)
    # Indirect(w) = (a1 + a3*w) * (b1 + b3*w)
    # -------------------------
    indirect_low = round((a1 + a3 * w_low) * (b1 + b3 * w_low), 4)
    indirect_high = round((a1 + a3 * w_high) * (b1 + b3 * w_high), 4)

    boot_low = []
    boot_high = []
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        boot = sub.iloc[idx].copy()

        # a-model bootstrap
        boot["XW_b"] = boot[x_col] * boot[w_col]
        Xa_b = sm.add_constant(boot[[x_col, w_col, "XW_b"]])
        a_b = sm.OLS(boot[m_col], Xa_b).fit()

        # y-model bootstrap
        boot["MW_b"] = boot[m_col] * boot[w_col]
        Xy_b = sm.add_constant(boot[[x_col, m_col, w_col, "MW_b"]])
        y_b = sm.OLS(boot[y_col], Xy_b).fit()

        a1_b = a_b.params[x_col]
        a3_b = a_b.params["XW_b"]
        b1_b = y_b.params[m_col]
        b3_b = y_b.params["MW_b"]

        boot_low.append((a1_b + a3_b * w_low) * (b1_b + b3_b * w_low))
        boot_high.append((a1_b + a3_b * w_high) * (b1_b + b3_b * w_high))

    boot_low = np.array(boot_low)
    boot_high = np.array(boot_high)

    ci_low_lo, ci_low_hi = np.percentile(boot_low, [2.5, 97.5]).round(4)
    ci_high_lo, ci_high_hi = np.percentile(boot_high, [2.5, 97.5]).round(4)

    se_low = round(boot_low.std(), 4)
    se_high = round(boot_high.std(), 4)

    sig_low = "Significant" if (ci_low_lo > 0 or ci_low_hi < 0) else "Not significant"
    sig_high = "Significant" if (ci_high_lo > 0 or ci_high_hi < 0) else "Not significant"

    # ========================================================
    # Tables
    # ========================================================

    results_paths = pd.DataFrame({
        "Path": [
            "Total Effect (c)",
            "Direct Effect (c')",
            "Path a1 (X → M)",
            "Path a2 (W → M)",
            "Path a3 (X×W → M)",
            "Path b1 (M → Y | X,W,MW)",
            "Path b2 (W → Y | X,M,MW)",
            "Path b3 (M×W → Y)",
        ],
        "Coefficient (B)": [c, cprime, a1, a2, a3, b1, b2, b3],
        "SE": [c_se, cprime_se, a1_se, a2_se, a3_se, b1_se, b2_se, b3_se],
        "t-value": [c_t, cprime_t, a1_t, a2_t, a3_t, b1_t, b2_t, b3_t],
        "p-value": [c_p, cprime_p, a1_p, a2_p, a3_p, b1_p, b2_p, b3_p],
        "95% CI (LL)": [c_ll, cprime_ll, a1_ll, a2_ll, a3_ll, b1_ll, b2_ll, b3_ll],
        "95% CI (UL)": [c_ul, cprime_ul, a1_ul, a2_ul, a3_ul, b1_ul, b2_ul, b3_ul],
    })
    results_paths["Significance"] = results_paths["p-value"].apply(sig_label)

    results_indirect = pd.DataFrame({
        "Moderator_Value": [w_low, w_high],
        "Moderator_Label": [f"{w_col}={w_low}", f"{w_col}={w_high}"],
        "Effect (a,b-path moderated)": [indirect_low, indirect_high],
        "Boot SE": [se_low, se_high],
        "BootLLCI": [ci_low_lo, ci_high_lo],
        "BootULCI": [ci_low_hi, ci_high_hi],
        "Significance": [sig_low, sig_high],
    })

    path_outfile = os.path.join(out_dir, f"{outfile_prefix}_paths.csv")
    indirect_outfile = os.path.join(out_dir, f"{outfile_prefix}_indirect.csv")

    results_paths.to_csv(path_outfile, float_format="%.4f", index=False)
    results_indirect.to_csv(indirect_outfile, float_format="%.4f", index=False)

    print(f"Saved path table     → {path_outfile}")
    print(f"Saved indirect table → {indirect_outfile}")

    # ========================================================
    # APA-style console output
    # ========================================================
    print("\nAPA-style summary")
    print("-----------------")
    print(f"Mediator: {m_col}")
    print(f"n = {n}")

    print(
        f"Total effect (c): b = {c:.4f}, SE = {c_se:.4f}, "
        f"t({int(model_c.df_resid)}) = {c_t:.4f}, p {format_p(c_p)}, "
        f"95% CI [{c_ll:.4f}, {c_ul:.4f}]"
    )
    print(
        f"Direct effect (c'): b = {cprime:.4f}, SE = {cprime_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {cprime_t:.4f}, p {format_p(cprime_p)}, "
        f"95% CI [{cprime_ll:.4f}, {cprime_ul:.4f}]"
    )

    print(
        f"Path a1 (X → M): b = {a1:.4f}, SE = {a1_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a1_t:.4f}, p {format_p(a1_p)}, "
        f"95% CI [{a1_ll:.4f}, {a1_ul:.4f}]"
    )
    print(
        f"Path a2 (W → M): b = {a2:.4f}, SE = {a2_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a2_t:.4f}, p {format_p(a2_p)}, "
        f"95% CI [{a2_ll:.4f}, {a2_ul:.4f}]"
    )
    print(
        f"Path a3 (X×W → M): b = {a3:.4f}, SE = {a3_se:.4f}, "
        f"t({int(model_a.df_resid)}) = {a3_t:.4f}, p {format_p(a3_p)}, "
        f"95% CI [{a3_ll:.4f}, {a3_ul:.4f}]"
    )

    print(
        f"Path b1 (M → Y | X,W,MW): b = {b1:.4f}, SE = {b1_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b1_t:.4f}, p {format_p(b1_p)}, "
        f"95% CI [{b1_ll:.4f}, {b1_ul:.4f}]"
    )
    print(
        f"Path b2 (W → Y | X,M,MW): b = {b2:.4f}, SE = {b2_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b2_t:.4f}, p {format_p(b2_p)}, "
        f"95% CI [{b2_ll:.4f}, {b2_ul:.4f}]"
    )
    print(
        f"Path b3 (M×W → Y): b = {b3:.4f}, SE = {b3_se:.4f}, "
        f"t({int(model_y.df_resid)}) = {b3_t:.4f}, p {format_p(b3_p)}, "
        f"95% CI [{b3_ll:.4f}, {b3_ul:.4f}]"
    )

    print("\nConditional indirect effects (a- and b-path moderated):")
    print(
        f"- W = {w_low}: "
        f"Effect = {indirect_low:.4f}, Boot SE = {se_low:.4f}, "
        f"95% CI [{ci_low_lo:.4f}, {ci_low_hi:.4f}] → {sig_low}"
    )
    print(
        f"- W = {w_high}: "
        f"Effect = {indirect_high:.4f}, Boot SE = {se_high:.4f}, "
        f"95% CI [{ci_high_lo:.4f}, {ci_high_hi:.4f}] → {sig_high}"
    )

    return results_paths, results_indirect


import os
import pandas as pd

# ============================================================
# PROCESS Model 15 script: Mall Donation Money × Awe
# ============================================================

INPUT_PATH = "../../../Merging/Results/master_with_dv_means_recoded.csv"

BASE_OUT = "Tables/Mall/Donation_Money_Awe"
OUTPUT_DIR_MODEL15 = os.path.join(BASE_OUT, "Model15")

os.makedirs(OUTPUT_DIR_MODEL15, exist_ok=True)

X_COL = "Style_Glam0_Hippie1"
Y_COL = "DonationMoney"
VIDEO_COL = "Awe_3cat"

STATUS_OLD = "Social_Status_Old_1_4"
STATUS_NEW = "Social_Status_New_1_7"

MALL_EXPERIMENTS = ["M_D_M", "M_D_N_M", "M_D_A_M"]

data = pd.read_csv(INPUT_PATH)
mall_dm = data[data["Experiment_Type"].isin(MALL_EXPERIMENTS)].copy()

# Map Experiment_Type → Awe_3cat
exp_to_awe = {"M_D_M": 0, "M_D_N_M": 1, "M_D_A_M": 2}
mall_dm[VIDEO_COL] = mall_dm["Experiment_Type"].map(exp_to_awe)

print("▶ Loaded master data for Model 15")
print(f"Mall donation-money subset n = {len(mall_dm)}")
print("\nExperiment_Type in mall_dm:")
print(mall_dm["Experiment_Type"].value_counts(dropna=False))
print("\nAwe_3cat in mall_dm after recode:")
print(mall_dm[VIDEO_COL].value_counts(dropna=False))

PAIR_DEFS = [
    ("0_1", "NoVideo_vs_Neutral", ["M_D_M", "M_D_N_M"]),
    ("1_2", "Neutral_vs_Awe",     ["M_D_N_M", "M_D_A_M"]),
    ("0_2", "NoVideo_vs_Awe",     ["M_D_M", "M_D_A_M"]),
]


def prepare_pair_subset(df, exp_types, pair_id, pair_label):
    sub = df[df["Experiment_Type"].isin(exp_types)].copy()
    print(f"\n▶ Preparing pair {pair_id}: {pair_label}")
    print("Experiment_Type counts:")
    print(sub["Experiment_Type"].value_counts(dropna=False))
    print("Awe_3cat counts:")
    print(sub[VIDEO_COL].value_counts(dropna=False))
    return sub


for pair_id, pair_label, exp_types in PAIR_DEFS:
    pair_df = prepare_pair_subset(mall_dm, exp_types, pair_id, pair_label)

    pair_out_dir = os.path.join(OUTPUT_DIR_MODEL15, f"Pair_{pair_id}_{pair_label}")
    os.makedirs(pair_out_dir, exist_ok=True)

    print(f"\n▶ Model 15 – Status_New, pair {pair_id} ({pair_label})")
    run_process_model15_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_NEW,
        y_col=Y_COL,
        w_col=VIDEO_COL,
        out_dir=pair_out_dir,
        outfile_prefix=f"StatusNew_{pair_label}_DonationMoney",
    )

    print(f"\n▶ Model 15 – Status_Old, pair {pair_id} ({pair_label})")
    run_process_model15_and_export(
        data=pair_df,
        x_col=X_COL,
        m_col=STATUS_OLD,
        y_col=Y_COL,
        w_col=VIDEO_COL,
        out_dir=pair_out_dir,
        outfile_prefix=f"StatusOld_{pair_label}_DonationMoney",
    )

print("\n✅ Finished PROCESS Model 15 analyses for DonationMoney × Status (Old + New) with Awe moderating a- and b-path.")

